<a href="https://colab.research.google.com/github/bountyhunter12/SemEval-2026-Task-10-Psycholinguistic-Conspiracy-Marker-Extraction-and-Detection/blob/main/Conspiracy_Marker_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
faoziafariha_train_redacted_path = kagglehub.dataset_download('faoziafariha/train-redacted')
faoziafariha_dev_redacted_path = kagglehub.dataset_download('faoziafariha/dev-redacted')
faoziafariha_test_reducted_path = kagglehub.dataset_download('faoziafariha/test-reducted')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-redacted/train_redacted.jsonl
/kaggle/input/dev-redacted/dev_redacted.jsonl
/kaggle/input/test-reducted/test_rehydrated.jsonl


In [ ]:
!pip install -U transformers datasets scikit-learn accelerate beautifulsoup4 markdown tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 109.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: markdown
    Found existing installation: Markdown 3.9
    Uninstalling Markdown-3.9:
      

In [ ]:
# Clean up old models/files to free space
import shutil
import os

# Remove old results
for folder in ['./results', './marker_results', './ensemble']:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Cleaned: {folder}")

# Clean Kaggle working directory
import glob
for f in glob.glob('/kaggle/working/*'):
    try:
        os.remove(f)
        print(f"Removed: {f}")
    except:
        pass

print("✅ Disk space freed!")

✅ Disk space freed!


In [ ]:
import requests
from tqdm import tqdm
import json
import re
import time
from bs4 import BeautifulSoup
from markdown import markdown

In [ ]:
!pip install seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
"""
IMPROVED REHYDRATION & PREPROCESSING
Better data quality = Better model performance
"""
import json
import re
import time
import requests
from bs4 import BeautifulSoup
from markdown import markdown
from tqdm import tqdm

# ===========================
# IMPROVED PREPROCESSING
# ===========================

def markdown_to_text(markdown_string):
    """Convert markdown to clean text - IMPROVED"""
    # Handle code blocks first
    html = markdown(markdown_string)
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html, flags=re.DOTALL)
    html = re.sub(r'<code>(.*?)</code>', ' ', html)

    soup = BeautifulSoup(html, "html.parser")
    text = ' '.join(soup.find_all(string=True))

    # Clean up excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def replace_urls(text, url_token='[URL]'):
    """Replace URLs but preserve their positions better"""
    # Match various URL formats
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, url_token, text)

def replace_ss_prefix(text):
    """Remove submission statement prefixes"""
    patterns = [
        r'^\W*(summary statement|submission statement|ss)[^a-zA-Z]*',
        r'^\W*(s\.s\.|s/s)[^a-zA-Z]*',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.I | re.U)
    return text.strip()

def clean_special_chars(text):
    """Clean special characters while preserving structure"""
    # Remove excessive punctuation
    text = re.sub(r'([!?.]){3,}', r'\1\1', text)
    # Normalize quotes (using unicode escapes)
    text = text.replace('\u201c', '"').replace('\u201d', '"')  # Smart double quotes
    text = text.replace('\u2018', "'").replace('\u2019', "'")  # Smart single quotes
    # Remove zero-width characters
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    return text

def normalize_whitespace(text):
    """Normalize whitespace while preserving paragraph structure"""
    # Replace multiple newlines with double newline
    text = re.sub(r'\n{3,}', '\n\n', text)
    # Replace tabs and multiple spaces
    text = re.sub(r'[ \t]+', ' ', text)
    # Clean up spaces around newlines
    text = re.sub(r' *\n *', '\n', text)
    return text.strip()

def preprocess_improved(text):
    """
    IMPROVED preprocessing pipeline
    Order matters: markdown -> urls -> ss -> special chars -> whitespace
    """
    if not text or not isinstance(text, str):
        return ""

    # Step 1: Convert markdown
    text = markdown_to_text(text)

    # Step 2: Replace URLs
    text = replace_urls(text)

    # Step 3: Remove SS prefix
    text = replace_ss_prefix(text)

    # Step 4: Clean special characters
    text = clean_special_chars(text)

    # Step 5: Normalize whitespace
    text = normalize_whitespace(text)

    return text

# ===========================
# IMPROVED REHYDRATION
# ===========================

def fetch_from_arctic_shift_improved(comment_ids, max_retries=3, timeout=20):
    """Improved Arctic Shift fetching with better error handling"""
    url = "https://arctic-shift.photon-reddit.com/api/comments/ids"

    # Split into smaller batches for reliability
    batch_size = 10
    all_results = {}

    for i in range(0, len(comment_ids), batch_size):
        batch = comment_ids[i:i + batch_size]
        params = {
            "ids": ",".join(batch),
            "fields": "body,subreddit,id,author,created_utc"
        }

        for attempt in range(max_retries):
            try:
                with requests.Session() as session:
                    session.headers.update({
                        'Connection': 'close',
                        'User-Agent': 'Mozilla/5.0'
                    })
                    response = session.get(url, params=params, timeout=timeout)
                    response.raise_for_status()
                    data = response.json()

                    if "data" in data and isinstance(data["data"], list):
                        for item in data["data"]:
                            body = item.get('body', '').strip()
                            # More thorough deletion check
                            if (body and
                                body not in ['[deleted]', '[removed]', '[removed by moderator]'] and
                                len(body) > 10):  # Minimum length check
                                all_results[item['id']] = item
                        break
            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                else:
                    print(f"Failed batch after {max_retries} attempts")

        time.sleep(0.5)  # Rate limiting

    return all_results

def fetch_from_pushshift_improved(comment_ids, max_retries=2):
    """Improved Pushshift with better filtering"""
    results = {}

    for comment_id in comment_ids[:3]:  # Limit to avoid rate limiting
        url = "https://api.pullpush.io/reddit/comment/search"
        params = {"ids": comment_id}

        for attempt in range(max_retries):
            try:
                response = requests.get(url, params=params, timeout=15)
                response.raise_for_status()
                data = response.json()

                if "data" in data and len(data["data"]) > 0:
                    item = data["data"][0]
                    body = item.get('body', '').strip()

                    # Strict filtering
                    if (body and
                        body not in ['[deleted]', '[removed]'] and
                        len(body) > 10 and
                        not body.startswith('[removed')):
                        results[comment_id] = {
                            'id': comment_id,
                            'body': item['body'],
                            'subreddit': item.get('subreddit', 'unknown'),
                            'author': item.get('author', 'unknown'),
                            'created_utc': item.get('created_utc', 0)
                        }
                break
            except:
                if attempt < max_retries - 1:
                    time.sleep(1)

        time.sleep(0.5)

    return results

def rehydrate_comments_improved(input_file, output_file):
    """
    IMPROVED rehydration with better quality control
    """
    print("=" * 70)
    print(f"📂 IMPROVED REHYDRATION: {input_file}")
    print("=" * 70)

    # Load items
    items_to_process = []
    original_data_map = {}

    with open(input_file, encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line)
                if '_id' in item and item['_id'].startswith('t1_'):
                    items_to_process.append(item['_id'])
                    original_data_map[item['_id']] = item
            except:
                pass

    print(f"📊 Total comments to rehydrate: {len(items_to_process)}")

    # Process in batches
    batch_size = 20
    rehydrated_data = []
    deleted_count = 0
    low_quality_count = 0

    for i in tqdm(range(0, len(items_to_process), batch_size), desc="Rehydrating"):
        batch_full_ids = items_to_process[i:i + batch_size]
        batch_clean_ids = [x[3:] for x in batch_full_ids]

        # Try Arctic Shift first
        rehydrated_map = fetch_from_arctic_shift_improved(batch_clean_ids)

        # Fallback to Pushshift for missing items
        missing = [x for x in batch_clean_ids if x not in rehydrated_map]
        if missing and len(missing) <= 5:
            pushshift_results = fetch_from_pushshift_improved(missing)
            rehydrated_map.update(pushshift_results)

        # Process results
        for fid in batch_full_ids:
            cid = fid[3:]

            if cid in rehydrated_map:
                r = rehydrated_map[cid]
                o = original_data_map[fid]

                # Preprocess text
                preprocessed_text = preprocess_improved(r["body"])

                # Quality check
                if len(preprocessed_text) < 20:
                    low_quality_count += 1
                    continue

                rehydrated_data.append({
                    "_id": fid,
                    "text": preprocessed_text,
                    "subreddit": r.get("subreddit", "unknown"),
                    "conspiracy": o.get("conspiracy"),
                    "markers": o.get("markers", []),
                    "annotator": o.get("annotator"),
                    # Preserve metadata for analysis
                    "original_length": len(r["body"]),
                    "processed_length": len(preprocessed_text)
                })
            else:
                deleted_count += 1

        time.sleep(1)  # Rate limiting

    # Save results
    with open(output_file, "w", encoding="utf-8") as f:
        for x in rehydrated_data:
            f.write(json.dumps(x, ensure_ascii=False) + "\n")

    # Statistics
    print(f"\n{'='*70}")
    print(f"📊 REHYDRATION STATISTICS")
    print(f"{'='*70}")
    print(f"✅ Successfully rehydrated: {len(rehydrated_data)}")
    print(f"🗑️  Deleted/removed: {deleted_count}")
    print(f"⚠️  Low quality (skipped): {low_quality_count}")
    print(f"📈 Success rate: {len(rehydrated_data) / len(items_to_process) * 100:.2f}%")

    if rehydrated_data:
        avg_length = sum(x['processed_length'] for x in rehydrated_data) / len(rehydrated_data)
        print(f"📏 Average text length: {avg_length:.0f} characters")

    print(f"{'='*70}\n")

# ===========================
# RUN REHYDRATION
# ===========================

if __name__ == "__main__":
    # Rehydrate training data
    rehydrate_comments_improved(
        "/kaggle/input/train-redacted/train_redacted.jsonl",
        "train_rehydrated_v2.jsonl"
    )

    # Rehydrate dev data
    rehydrate_comments_improved(
        "/kaggle/input/dev-redacted/dev_redacted.jsonl",
        "dev_rehydrated_v2.jsonl"
    )

    # Rehydrate test data
    rehydrate_comments_improved(
        "/kaggle/input/test-reducted/test_rehydrated.jsonl",
        "test_rehydrated_v2.jsonl"
    )

    print("\n✅ All rehydration complete!")

📂 IMPROVED REHYDRATION: /kaggle/input/train-redacted/train_redacted.jsonl
📊 Total comments to rehydrate: 4361


Rehydrating: 100%|██████████| 219/219 [11:27<00:00,  3.14s/it]



📊 REHYDRATION STATISTICS
✅ Successfully rehydrated: 4316
🗑️  Deleted/removed: 45
⚠️  Low quality (skipped): 0
📈 Success rate: 98.97%
📏 Average text length: 415 characters

📂 IMPROVED REHYDRATION: /kaggle/input/dev-redacted/dev_redacted.jsonl
📊 Total comments to rehydrate: 100


Rehydrating: 100%|██████████| 5/5 [00:14<00:00,  2.90s/it]



📊 REHYDRATION STATISTICS
✅ Successfully rehydrated: 100
🗑️  Deleted/removed: 0
⚠️  Low quality (skipped): 0
📈 Success rate: 100.00%
📏 Average text length: 418 characters

📂 IMPROVED REHYDRATION: /kaggle/input/test-reducted/test_rehydrated.jsonl
📊 Total comments to rehydrate: 938


Rehydrating: 100%|██████████| 47/47 [02:16<00:00,  2.90s/it]


📊 REHYDRATION STATISTICS
✅ Successfully rehydrated: 938
🗑️  Deleted/removed: 0
⚠️  Low quality (skipped): 0
📈 Success rate: 100.00%
📏 Average text length: 419 characters


✅ All rehydration complete!


In [ ]:
"""
SUBTASK 1: IMPROVED FEW-SHOT MARKER EXTRACTION
Uses Qwen2.5-7B with optimized prompts
Target: 0.30+ F1 score
"""
import json
import os
import zipfile
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import re
from collections import defaultdict
import gc

# ===========================
# CONFIGURATION
# ===========================
INPUT_FILE = "/kaggle/input/test-reducted/test_rehydrated.jsonl"
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "/kaggle/working"
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "submission.jsonl")
OUTPUT_ZIP = os.path.join(OUTPUT_DIR, "submission_markers.zip")

MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

print("="*70)
print("SUBTASK 1: IMPROVED FEW-SHOT MARKER EXTRACTION")
print("="*70)

# ===========================
# ENHANCED FEW-SHOT EXAMPLES
# ===========================
FEW_SHOT_EXAMPLES = [
    {
        "text": "SS: Just a reminder that various extremist groups that are out there, trying to recruit people, are actually CIA and other Alphabet agency Honeypots. The BBC is even admitting this guy is CIA but says he's only 'former' CIA (yea right) and they are making sure to create a link to Russia so they can continue with the narrative that anyone who isn't on the left is a Russian controlled Nazi.",
        "markers": [
            {"type": "Actor", "text": "CIA"},
            {"type": "Actor", "text": "other Alphabet agency"},
            {"type": "Action", "text": "Honeypots"},
            {"type": "Evidence", "text": "The BBC is even admitting this guy is CIA"},
            {"type": "Action", "text": "continue with the narrative"}
        ]
    },
    {
        "text": "New report alleges Hillary Clinton oversaw a multi-billion dollar fraud/theft, and high-ranking FBI agents are now coming forward with more details about it.",
        "markers": [
            {"type": "Actor", "text": "Hillary Clinton"},
            {"type": "Action", "text": "oversaw a multi-billion dollar fraud/theft"},
            {"type": "Evidence", "text": "high-ranking FBI agents are now coming forward"}
        ]
    },
    {
        "text": "Maxwell claims that her email server was hacked after a court unsealed approximately 2,000 pages of documents. If emails were obtained in the hack they could showcase embarrassing information on Epstein's clients, alleged victims, and co-conspirators in his massive sex trafficking operation.",
        "markers": [
            {"type": "Action", "text": "email server was hacked"},
            {"type": "Effect", "text": "could showcase embarrassing information"},
            {"type": "Victim", "text": "Epstein's clients, alleged victims"},
            {"type": "Effect", "text": "sex trafficking operation"}
        ]
    },
    {
        "text": "Rudy drops radical bombs on Fox and Friends implicating the highest levels of government, the ambassadors, and intelligence agencies. He makes names and has recorded testimony from individuals banned from the United States to keep it quiet.",
        "markers": [
            {"type": "Actor", "text": "highest levels of government"},
            {"type": "Actor", "text": "ambassadors"},
            {"type": "Actor", "text": "intelligence agencies"},
            {"type": "Victim", "text": "individuals banned from the United States"},
            {"type": "Action", "text": "to keep it quiet"}
        ]
    },
    {
        "text": "The government is covering up evidence of vaccine injuries to protect pharmaceutical companies. Thousands of people have died but mainstream media refuses to report it because they're paid off by Big Pharma.",
        "markers": [
            {"type": "Actor", "text": "The government"},
            {"type": "Actor", "text": "pharmaceutical companies"},
            {"type": "Actor", "text": "mainstream media"},
            {"type": "Actor", "text": "Big Pharma"},
            {"type": "Action", "text": "covering up evidence"},
            {"type": "Effect", "text": "vaccine injuries"},
            {"type": "Victim", "text": "Thousands of people have died"},
            {"type": "Evidence", "text": "refuses to report it because they're paid off"}
        ]
    }
]

# ===========================
# IMPROVED PROMPT ENGINEERING
# ===========================
def create_extraction_prompt(text):
    """Create a powerful few-shot prompt with clear instructions"""

    system_prompt = """You are an expert at analyzing conspiracy theory text and identifying psycholinguistic markers.

MARKER DEFINITIONS:
1. Actor: Who is allegedly responsible? (groups, individuals, organizations with malicious intent)
   Examples: "CIA", "Big Pharma", "the government", "mainstream media"

2. Action: What malicious plans or covert operations are they doing?
   Examples: "covering up evidence", "manipulating elections", "poisoning the water"

3. Effect: What are the negative consequences or outcomes?
   Examples: "thousands died", "economic collapse", "loss of freedoms"

4. Victim: Who is being targeted or harmed?
   Examples: "innocent citizens", "children", "whistleblowers"

5. Evidence: How does the author support their claims?
   Examples: "leaked documents show", "experts are warning", "too many coincidences"

RULES:
- Extract EXACT text spans (do not paraphrase)
- Multiple markers can overlap
- Extract ALL relevant spans, even if short
- If unclear or no markers exist, return empty list"""

    # Build examples with clear formatting
    examples_text = ""
    for i, ex in enumerate(FEW_SHOT_EXAMPLES, 1):
        examples_text += f"\n### EXAMPLE {i} ###\n"
        examples_text += f"Text: {ex['text']}\n\n"
        examples_text += "Markers:\n"
        for m in ex['markers']:
            examples_text += f'- {m["type"]}: "{m["text"]}"\n'
        examples_text += "\n"

    user_prompt = f"""{examples_text}

### YOUR TASK ###
Extract markers from this text:

Text: {text}

Respond with ONLY a JSON array (no markdown, no explanation):
[
  {{"type": "Actor", "text": "exact span"}},
  {{"type": "Action", "text": "exact span"}}
]

If no markers: []

JSON output:"""

    return system_prompt, user_prompt

# ===========================
# LOAD MODEL
# ===========================
def load_llm():
    """Load Qwen model"""
    print("\n📦 Loading LLM...")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )

    print(f"✅ Loaded {MODEL_NAME}")
    return tokenizer, model, device

# ===========================
# EXTRACT MARKERS - FIXED
# ===========================
def extract_markers_llm(text, tokenizer, model):
    """Extract markers using LLM"""

    system_prompt, user_prompt = create_extraction_prompt(text)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Tokenize properly
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        text_input,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    ).to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.1,  # Lower temperature for more consistent output
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
        )

    # Decode only new tokens
    generated_ids = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)

    return response

# ===========================
# PARSE LLM OUTPUT - ENHANCED
# ===========================
def parse_markers_from_response(response, text):
    """Parse JSON markers and find positions"""

    try:
        # Clean response
        response = response.strip()

        # Extract JSON - try multiple patterns
        json_match = re.search(r'\[.*?\]', response, re.DOTALL)
        if not json_match:
            # Try to find it after "JSON output:" or similar
            if "JSON output:" in response:
                response = response.split("JSON output:")[-1]
                json_match = re.search(r'\[.*?\]', response, re.DOTALL)

        if not json_match:
            return []

        json_str = json_match.group(0)
        markers_list = json.loads(json_str)

        if not isinstance(markers_list, list):
            return []

        # Find positions for each marker
        result_markers = []
        for marker in markers_list:
            if not isinstance(marker, dict):
                continue

            marker_text = marker.get('text', '').strip()
            marker_type = marker.get('type', '')

            if not marker_text or marker_type not in MARKER_TYPES:
                continue

            # Find all occurrences
            start_idx = 0
            found_positions = []
            while True:
                pos = text.find(marker_text, start_idx)
                if pos == -1:
                    break
                found_positions.append(pos)
                start_idx = pos + 1

            # Add all found positions
            for pos in found_positions:
                result_markers.append({
                    'startIndex': pos,
                    'endIndex': pos + len(marker_text),
                    'type': marker_type,
                    'text': marker_text
                })

        # Remove duplicates
        seen = set()
        unique_markers = []
        for m in result_markers:
            key = (m['startIndex'], m['endIndex'], m['type'])
            if key not in seen:
                seen.add(key)
                unique_markers.append(m)

        return unique_markers

    except Exception as e:
        # Silent fail - return empty list
        return []

# ===========================
# PROCESS DOCUMENTS
# ===========================
def process_document(item, tokenizer, model):
    """Process single document"""
    doc_id = item.get('_id')
    text = item.get('text', '').strip()

    if not doc_id or not text or len(text) < 20:
        return {"_id": doc_id, "markers": []}

    try:
        # Extract with LLM
        response = extract_markers_llm(text, tokenizer, model)

        # Parse markers
        markers = parse_markers_from_response(response, text)

        return {
            "_id": doc_id,
            "markers": markers
        }
    except Exception as e:
        return {"_id": doc_id, "markers": []}

# ===========================
# MAIN EXECUTION
# ===========================
def main():
    print("\n" + "="*70)
    print("STARTING FEW-SHOT MARKER EXTRACTION")
    print("="*70)

    # Load model
    tokenizer, model, device = load_llm()

    # Load test data
    print(f"\n📂 Loading: {INPUT_FILE}")

    input_data = []
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                input_data.append(json.loads(line))
            except:
                continue

    print(f"✅ Loaded {len(input_data)} documents")

    # Process documents
    print(f"\n🔄 Extracting markers...")
    predictions = []

    for item in tqdm(input_data, desc="Processing"):
        pred = process_document(item, tokenizer, model)
        predictions.append(pred)

        # Free memory periodically
        if len(predictions) % 50 == 0:
            torch.cuda.empty_cache()
            gc.collect()

    # Statistics
    total_markers = sum(len(p['markers']) for p in predictions)
    docs_with_markers = sum(1 for p in predictions if len(p['markers']) > 0)

    print(f"\n📊 EXTRACTION STATS:")
    print(f"   Documents processed: {len(predictions)}")
    print(f"   Total markers: {total_markers}")
    print(f"   Docs with markers: {docs_with_markers} ({docs_with_markers/len(predictions)*100:.1f}%)")
    print(f"   Avg markers/doc: {total_markers/len(predictions):.2f}")

    # Marker distribution
    marker_counts = defaultdict(int)
    for pred in predictions:
        for marker in pred['markers']:
            marker_counts[marker['type']] += 1

    if total_markers > 0:
        print(f"\n📊 Marker Distribution:")
        for mtype in MARKER_TYPES:
            count = marker_counts[mtype]
            pct = (count / total_markers * 100) if total_markers > 0 else 0
            print(f"   {mtype:10s}: {count:5d} ({pct:5.2f}%)")

    # Save
    print(f"\n💾 Saving to {OUTPUT_FILE}...")
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        for pred in predictions:
            f.write(json.dumps(pred) + '\n')

    # Create ZIP
    print(f"📦 Creating {OUTPUT_ZIP}...")
    with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(OUTPUT_FILE, arcname="submission.jsonl")

    os.remove(OUTPUT_FILE)

    print(f"\n✅ SUCCESS! File: {OUTPUT_ZIP}")

    # Sample output
    print(f"\n📋 SAMPLE PREDICTIONS:")
    for i, pred in enumerate(predictions[:3], 1):
        print(f"\n{i}. Doc: {pred['_id']}")
        if pred['markers']:
            for j, m in enumerate(pred['markers'][:5], 1):
                text_preview = m['text'][:60]
                if len(m['text']) > 60:
                    text_preview += "..."
                print(f"   {j}. [{m['type']:8s}] \"{text_preview}\"")
            if len(pred['markers']) > 5:
                print(f"   ... and {len(pred['markers']) - 5} more")
        else:
            print("   (No markers)")

    print("\n" + "="*70)

if __name__ == "__main__":
    main()

SUBTASK 1: IMPROVED FEW-SHOT MARKER EXTRACTION

STARTING FEW-SHOT MARKER EXTRACTION

📦 Loading LLM...
Device: cuda


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


✅ Loaded Qwen/Qwen2.5-7B-Instruct

📂 Loading: /kaggle/input/test-reducted/test_rehydrated.jsonl
✅ Loaded 938 documents

🔄 Extracting markers...


Processing:  12%|█▏        | 114/938 [1:16:54<4:49:21, 21.07s/it]  

Keep

In [ ]:
# """
# DeBERTa-v3-Large Training - DISK SPACE OPTIMIZED
# Fixes: No space left on device
# """
# import json
# import torch
# import numpy as np
# import os
# import shutil
# import gc
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForTokenClassification,
#     Trainer,
#     TrainingArguments,
#     DataCollatorForTokenClassification,
#     EarlyStoppingCallback
# )
# from seqeval.metrics import f1_score as seqeval_f1
# from seqeval.scheme import IOB2
# from collections import Counter
# import warnings
# warnings.filterwarnings('ignore')
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # ===========================
# # CLEANUP OLD FILES
# # ===========================
# print("="*70)
# print("CLEANING UP DISK SPACE")
# print("="*70)

# # Remove old model directories
# old_dirs = [
#     "./results",
#     "./marker_results",
#     "./marker_results_advanced",
#     "./marker_results_large",
#     "./model_deberta_large",
#     "./model_1_deberta_large",
#     "./model_2_xlm_large",
#     "./model_3_roberta_large"
# ]

# for old_dir in old_dirs:
#     if os.path.exists(old_dir):
#         try:
#             shutil.rmtree(old_dir)
#             print(f"✅ Removed: {old_dir}")
#         except:
#             pass

# # Clean working directory
# working_files = [
#     "/kaggle/working/submission.jsonl",
#     "/kaggle/working/submission.zip",
#     "/kaggle/working/submission_markers.zip"
# ]

# for f in working_files:
#     if os.path.exists(f):
#         try:
#             os.remove(f)
#             print(f"✅ Removed: {f}")
#         except:
#             pass

# gc.collect()
# torch.cuda.empty_cache() if torch.cuda.is_available() else None

# print("✅ Cleanup complete!\n")

# # ===========================
# # CONFIGURATION
# # ===========================
# print("="*70)
# print("DEBERTA-V3-LARGE TRAINING - SPACE OPTIMIZED")
# print("="*70)

# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# labels_list = ["O"]
# for marker_type in MARKER_TYPES:
#     labels_list.append(f"B-{marker_type}")
#     labels_list.append(f"I-{marker_type}")

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# print(f"\nLabels: {len(labels_list)} total")

# # ===========================
# # ALIGNMENT
# # ===========================
# def align_labels_advanced(text, markers, tokenizer, max_length=512):
#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=max_length,
#         padding='max_length',
#         return_offsets_mapping=True,
#         add_special_tokens=True
#     )

#     offset_mapping = encoding['offset_mapping']
#     labels = []

#     for token_start, token_end in offset_mapping:
#         if token_start == 0 and token_end == 0:
#             labels.append(-100)
#         else:
#             labels.append(label2id["O"])

#     sorted_markers = sorted(markers, key=lambda x: (x['startIndex'], x['endIndex']))

#     for marker in sorted_markers:
#         start_char = marker['startIndex']
#         end_char = marker['endIndex']
#         marker_type = marker['type']

#         if marker_type not in MARKER_TYPES:
#             continue

#         first_token_idx = None
#         last_token_idx = None

#         for idx, (token_start, token_end) in enumerate(offset_mapping):
#             if token_start == 0 and token_end == 0:
#                 continue

#             if token_start < end_char and token_end > start_char:
#                 overlap_start = max(token_start, start_char)
#                 overlap_end = min(token_end, end_char)
#                 overlap_ratio = (overlap_end - overlap_start) / (token_end - token_start)

#                 if overlap_ratio >= 0.4:
#                     if first_token_idx is None:
#                         first_token_idx = idx
#                     last_token_idx = idx

#         if first_token_idx is not None:
#             labels[first_token_idx] = label2id[f"B-{marker_type}"]

#             if last_token_idx is not None and last_token_idx > first_token_idx:
#                 for idx in range(first_token_idx + 1, last_token_idx + 1):
#                     if labels[idx] != -100:
#                         labels[idx] = label2id[f"I-{marker_type}"]

#     return labels

# def create_dataset_advanced(jsonl_path, tokenizer, max_length=512, min_length=30):
#     all_input_ids = []
#     all_attention_masks = []
#     all_labels = []

#     stats = {'total': 0, 'kept': 0, 'skipped_short': 0, 'skipped_no_markers': 0, 'total_markers': 0}

#     print(f"\nProcessing {jsonl_path}...")

#     with open(jsonl_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             stats['total'] += 1

#             try:
#                 item = json.loads(line)
#                 text = item.get('text', '').strip()
#                 markers = item.get('markers', [])

#                 if len(text) < min_length:
#                     stats['skipped_short'] += 1
#                     continue

#                 if 'train' in jsonl_path and len(markers) == 0:
#                     stats['skipped_no_markers'] += 1
#                     continue

#                 stats['total_markers'] += len(markers)

#                 encoding = tokenizer(text, truncation=True, max_length=max_length, padding='max_length')
#                 label_ids = align_labels_advanced(text, markers, tokenizer, max_length)

#                 all_input_ids.append(encoding['input_ids'])
#                 all_attention_masks.append(encoding['attention_mask'])
#                 all_labels.append(label_ids)
#                 stats['kept'] += 1

#             except:
#                 continue

#     print(f"✅ Loaded {stats['kept']} examples, {stats['total_markers']} markers")

#     return Dataset.from_dict({
#         'input_ids': all_input_ids,
#         'attention_mask': all_attention_masks,
#         'labels': all_labels
#     })

# # ===========================
# # LOAD DATA
# # ===========================
# print("\n" + "="*70)
# print("LOADING DATA")
# print("="*70)

# model_name = "microsoft/deberta-v3-large"
# print(f"Model: {model_name}")

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# train_dataset = create_dataset_advanced("train_rehydrated_v2.jsonl", tokenizer, max_length=512)
# val_dataset_temp = create_dataset_advanced("dev_rehydrated_v2.jsonl", tokenizer, max_length=512)

# val_has_markers = False
# if len(val_dataset_temp) > 0:
#     for i in range(min(10, len(val_dataset_temp))):
#         if any(l > 0 and l != -100 for l in val_dataset_temp[i]['labels']):
#             val_has_markers = True
#             break

# if not val_has_markers:
#     print("\n⚠️ Creating validation split (20%)")
#     split = train_dataset.train_test_split(test_size=0.2, seed=42)
#     train_dataset = split['train']
#     val_dataset = split['test']
#     print(f"✅ Split: {len(train_dataset)} train, {len(val_dataset)} val")
# else:
#     val_dataset = val_dataset_temp

# train_dataset.set_format("torch")
# val_dataset.set_format("torch")

# # ===========================
# # MODEL
# # ===========================
# print("\n" + "="*70)
# print("INITIALIZING MODEL")
# print("="*70)

# model = AutoModelForTokenClassification.from_pretrained(
#     model_name,
#     num_labels=len(labels_list),
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True
# )

# print(f"✅ DeBERTa-v3-Large loaded (304M parameters)")

# # ===========================
# # METRICS
# # ===========================
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=2)

#     true_labels = []
#     pred_labels = []

#     for prediction, label in zip(predictions, labels):
#         true_seq = []
#         pred_seq = []

#         for pred_id, label_id in zip(prediction, label):
#             if label_id != -100:
#                 true_seq.append(id2label[label_id])
#                 pred_seq.append(id2label[pred_id])

#         if len(true_seq) > 0:
#             true_labels.append(true_seq)
#             pred_labels.append(pred_seq)

#     results = {}

#     try:
#         results["overall_f1"] = seqeval_f1(true_labels, pred_labels, mode='strict', scheme=IOB2)
#     except:
#         results["overall_f1"] = 0.0

#     for marker_type in MARKER_TYPES:
#         filtered_true = []
#         filtered_pred = []

#         for true_seq, pred_seq in zip(true_labels, pred_labels):
#             ft = [t if marker_type in t else 'O' for t in true_seq]
#             fp = [p if marker_type in p else 'O' for p in pred_seq]
#             filtered_true.append(ft)
#             filtered_pred.append(fp)

#         try:
#             if any(any(x != 'O' for x in seq) for seq in filtered_true):
#                 results[f"f1_{marker_type}"] = seqeval_f1(filtered_true, filtered_pred, mode='strict', scheme=IOB2)
#             else:
#                 results[f"f1_{marker_type}"] = 0.0
#         except:
#             results[f"f1_{marker_type}"] = 0.0

#     return results

# # ===========================
# # TRAINING ARGS - SPACE OPTIMIZED (FIXED)
# # ===========================
# training_args = TrainingArguments(
#     output_dir="./deberta_large_final",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-6,
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=4,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     load_best_model_at_end=True,
#     metric_for_best_model="overall_f1",
#     greater_is_better=True,
#     logging_steps=100,
#     report_to="none",
#     save_total_limit=2,  # ⭐ CHANGED: Keep 2 (current + best) instead of 1
#     fp16=torch.cuda.is_available(),
#     gradient_accumulation_steps=8,
#     dataloader_num_workers=0,
#     lr_scheduler_type="cosine",
#     seed=42,
#     save_safetensors=False,
# )

# # ===========================
# # REMOVE THE CUSTOM CALLBACK - NOT NEEDED
# # ===========================
# # The SpaceSaverCallback was causing the issue by deleting checkpoints too early
# # save_total_limit=2 is enough

# # ===========================
# # TRAINER
# # ===========================
# data_collator = DataCollatorForTokenClassification(
#     tokenizer=tokenizer,
#     padding=True,
#     label_pad_token_id=-100
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # ⭐ REMOVED SpaceSaverCallback
# )

# # ===========================
# # TRAIN
# # ===========================
# print("\n" + "="*70)
# print("STARTING TRAINING")
# print("="*70)
# print("🎯 Target: F1 0.25-0.35")
# print("💾 Space-optimized: Only keeps best checkpoint")
# print("="*70 + "\n")

# trainer.train()

# # ===========================
# # SAVE FINAL MODEL
# # ===========================
# print("\n" + "="*70)
# print("SAVING FINAL MODEL")
# print("="*70)

# final_model_path = "./deberta_large_final/checkpoint-best"
# trainer.save_model(final_model_path)
# tokenizer.save_pretrained(final_model_path)

# print(f"✅ Saved to: {final_model_path}")

# # Clean up intermediate checkpoints
# for checkpoint_dir in os.listdir("./deberta_large_final"):
#     if checkpoint_dir.startswith("checkpoint-") and checkpoint_dir != "checkpoint-best":
#         full_path = os.path.join("./deberta_large_final", checkpoint_dir)
#         try:
#             shutil.rmtree(full_path)
#             print(f"🗑️  Cleaned: {full_path}")
#         except:
#             pass

# # ===========================
# # EVALUATION
# # ===========================
# print("\n" + "="*70)
# print("FINAL EVALUATION")
# print("="*70)

# eval_results = trainer.evaluate()

# print(f"\n📊 RESULTS:")
# print(f"   Overall F1: {eval_results['eval_overall_f1']:.4f}")

# print(f"\n📊 Per-Marker:")
# for marker_type in MARKER_TYPES:
#     key = f"eval_f1_{marker_type}"
#     if key in eval_results:
#         score = eval_results[key]
#         emoji = "🎉" if score > 0.35 else "✅" if score > 0.20 else "⚠️"
#         print(f"   {emoji} {marker_type:10s}: {score:.4f}")

# print("\n" + "="*70)
# print("TRAINING COMPLETE!")
# print("="*70)

# # Performance analysis
# base_f1 = 0.18
# improvement = ((eval_results['eval_overall_f1'] - base_f1) / base_f1) * 100

# print(f"\n📈 IMPROVEMENT:")
# print(f"   Base model F1: {base_f1:.4f}")
# print(f"   Large model F1: {eval_results['eval_overall_f1']:.4f}")
# print(f"   Gain: +{improvement:.1f}%")

# if eval_results['eval_overall_f1'] >= 0.30:
#     print(f"\n🎉 EXCELLENT! F1 > 0.30")
#     print(f"   Expected test F1: 0.25-0.35")
# elif eval_results['eval_overall_f1'] >= 0.25:
#     print(f"\n✅ GOOD! F1 > 0.25")
#     print(f"   Expected test F1: 0.22-0.30")
# else:
#     print(f"\n⚠️  Below target")
#     print(f"   Current: {eval_results['eval_overall_f1']:.4f}")
#     print(f"   Need: 0.25+")

# print("="*70) """

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [ ]:
!pip uninstall -y transformers accelerate peft sentence-transformers -q
!pip install transformers==4.41.2 accelerate==0.29.3 peft==0.10.0 sentence-transformers==5.1.1 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 7.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 8.8 MB/s eta 0:00:00a 0:00:01m


In [ ]:
"""
PSYCHOMARKER SUBTASK 1 - OPTIMIZED DEBERTA + LLM FEW-SHOT
Target: 0.18 → 0.35+ F1
"""

# ============================================================================
# FIXED IMPORTS (COMPATIBLE VERSIONS)
# ============================================================================
print("Installing compatible versions...")


!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install datasets==2.14.0 -q
!pip install seqeval==1.2.2 -q
!pip install openai -q  # For LLM API if available

import json
import torch
import numpy as np
import os
import re
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback
)
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
import warnings
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ Packages installed successfully")

# ============================================================================
# CONFIGURATION - OPTIMIZED
# ============================================================================

# Model selection - using base for speed, but larger is better
MODEL_NAME = "microsoft/deberta-v3-base"  # Changed from large for Kaggle memory
# Alternative: "microsoft/deberta-large" if you have memory

# Training parameters - CRITICAL FIXES
MAX_LENGTH = 256  # Reduced from 512 for efficiency
BATCH_SIZE = 8    # Increased from 2 (8x larger)
LEARNING_RATE = 3e-5  # Increased from 5e-6 (6x higher)
EPOCHS = 8        # Reduced from 10 but more effective
WARMUP_RATIO = 0.1
GRAD_ACCUM_STEPS = 4  # Reduced from 8

# Marker types
MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# Create label mapping
labels_list = ["O"]
for marker_type in MARKER_TYPES:
    labels_list.append(f"B-{marker_type}")
    labels_list.append(f"I-{marker_type}")

label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}

print("="*70)
print("OPTIMIZED MARKER EXTRACTION")
print("="*70)
print(f"\n📊 Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Batch size: {BATCH_SIZE} (was 2)")
print(f"  Learning rate: {LEARNING_RATE} (was 5e-6)")
print(f"  Max length: {MAX_LENGTH}")
print(f"  Epochs: {EPOCHS}")

# ============================================================================
# IMPROVED LABEL ALIGNMENT
# ============================================================================

def improved_align_labels(text, markers, tokenizer, max_length=MAX_LENGTH):
    """
    FIXED: Better label alignment with character-level matching
    """
    # Tokenize with offsets
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_offsets_mapping=True
    )

    offset_mapping = encoding['offset_mapping']
    labels = [-100] * len(offset_mapping)  # Initialize with -100

    # First pass: label all tokens as O (except special tokens)
    for i, (start, end) in enumerate(offset_mapping):
        if start == 0 and end == 0:
            continue  # Keep as -100 for padding
        else:
            labels[i] = label2id["O"]

    # Process each marker
    for marker in markers:
        start_char = marker['startIndex']
        end_char = marker['endIndex']
        marker_type = marker['type']

        if marker_type not in MARKER_TYPES:
            continue

        # Find tokens that overlap with this marker
        overlapping_indices = []

        for i, (token_start, token_end) in enumerate(offset_mapping):
            if token_start == 0 and token_end == 0:
                continue

            # Check for any overlap
            if not (token_end <= start_char or token_start >= end_char):
                overlapping_indices.append(i)

        if overlapping_indices:
            # Sort by position
            overlapping_indices.sort()

            # Label first token as B-
            first_idx = overlapping_indices[0]
            labels[first_idx] = label2id[f"B-{marker_type}"]

            # Label remaining tokens as I-
            for idx in overlapping_indices[1:]:
                labels[idx] = label2id[f"I-{marker_type}"]

    return labels

# ============================================================================
# DATA AUGMENTATION
# ============================================================================

def augment_marker_data(text, markers, augmentation_factor=1):
    """
    Create augmented versions of text with markers
    """
    augmented_examples = []

    # Original example
    augmented_examples.append((text, markers))

    if augmentation_factor > 0:
        # Augmentation 1: Synonym replacement for non-marker words
        words = text.split()
        if len(words) > 10:
            # Simple word swap augmentation
            import random
            non_marker_words = []

            # Identify non-marker words
            for i, word in enumerate(words):
                is_in_marker = False
                for marker in markers:
                    marker_text = marker.get('text', '')
                    if word in marker_text:
                        is_in_marker = True
                        break

                if not is_in_marker and len(word) > 3:
                    non_marker_words.append(i)

            if len(non_marker_words) >= 2:
                # Swap two non-marker words
                idx1, idx2 = random.sample(non_marker_words, 2)
                new_words = words.copy()
                new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
                new_text = ' '.join(new_words)

                # Adjust marker positions (simple approximation)
                new_markers = []
                for marker in markers:
                    old_text = marker['text']
                    if old_text in text:
                        # Find in new text
                        if old_text in new_text:
                            start_idx = new_text.find(old_text)
                            if start_idx >= 0:
                                new_marker = marker.copy()
                                new_marker['startIndex'] = start_idx
                                new_marker['endIndex'] = start_idx + len(old_text)
                                new_markers.append(new_marker)

                if new_markers:
                    augmented_examples.append((new_text, new_markers))

    return augmented_examples

# ============================================================================
# CREATE ENHANCED DATASET
# ============================================================================

def create_enhanced_dataset(jsonl_path, tokenizer, is_training=True, augmentation=True):
    """
    Create dataset with augmentation and better filtering
    """
    print(f"\n📂 Processing {jsonl_path}...")

    all_input_ids = []
    all_attention_masks = []
    all_labels = []

    stats = {
        'total': 0, 'kept': 0, 'skipped_short': 0,
        'skipped_no_markers': 0, 'total_markers': 0
    }

    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            stats['total'] += 1

            try:
                item = json.loads(line)
                text = item.get('text', '').strip()
                markers = item.get('markers', [])

                # Filter criteria
                if len(text) < 30 or len(text) > 2000:
                    stats['skipped_short'] += 1
                    continue

                # For training, skip examples without markers
                if is_training and len(markers) == 0:
                    stats['skipped_no_markers'] += 1
                    continue

                stats['total_markers'] += len(markers)

                # Apply augmentation for training
                if is_training and augmentation and len(markers) > 0:
                    examples = augment_marker_data(text, markers, augmentation_factor=1)
                else:
                    examples = [(text, markers)]

                for ex_text, ex_markers in examples:
                    # Tokenize
                    encoding = tokenizer(
                        ex_text,
                        truncation=True,
                        max_length=MAX_LENGTH,
                        padding='max_length',
                        return_tensors='pt'
                    )

                    # Align labels
                    label_ids = improved_align_labels(ex_text, ex_markers, tokenizer, MAX_LENGTH)

                    all_input_ids.append(encoding['input_ids'][0].tolist())
                    all_attention_masks.append(encoding['attention_mask'][0].tolist())
                    all_labels.append(label_ids)
                    stats['kept'] += 1

            except Exception as e:
                continue

    print(f"✅ Loaded {stats['kept']} examples, {stats['total_markers']} markers")
    print(f"   Skipped: {stats['skipped_short']} short, {stats['skipped_no_markers']} no markers")

    return Dataset.from_dict({
        'input_ids': all_input_ids,
        'attention_mask': all_attention_masks,
        'labels': all_labels
    })

# ============================================================================
# LOAD DATA
# ============================================================================

print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load datasets with augmentation
train_dataset = create_enhanced_dataset(
    "train_rehydrated_v2.jsonl",
    tokenizer,
    is_training=True,
    augmentation=True
)

val_dataset = create_enhanced_dataset(
    "dev_rehydrated_v2.jsonl",
    tokenizer,
    is_training=False,
    augmentation=False
)

# Split if validation is small
if len(val_dataset) < 50:
    print("\n⚠️  Validation set too small, splitting training data")
    split = train_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = split['train']
    val_dataset = split['test']

train_dataset.set_format("torch")
val_dataset.set_format("torch")

print(f"\n📊 Dataset sizes:")
print(f"  Training: {len(train_dataset)} examples")
print(f"  Validation: {len(val_dataset)} examples")

# ============================================================================
# INITIALIZE MODEL
# ============================================================================

print("\n" + "="*70)
print("INITIALIZING MODEL")
print("="*70)

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels_list),
    id2label=id2label,
    label2id=label2id
)

print(f"✅ Model loaded: {MODEL_NAME}")

# ============================================================================
# COMPUTE METRICS
# ============================================================================

def compute_enhanced_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    for i in range(len(labels)):
        true_seq = []
        pred_seq = []

        for j in range(len(labels[i])):
            if labels[i][j] != -100:
                true_seq.append(id2label[labels[i][j]])
                pred_seq.append(id2label[predictions[i][j]])

        if true_seq:
            true_labels.append(true_seq)
            pred_labels.append(pred_seq)

    results = {}

    # Calculate overall metrics
    try:
        results["overall_f1"] = f1_score(true_labels, pred_labels)
        results["overall_precision"] = precision_score(true_labels, pred_labels)
        results["overall_recall"] = recall_score(true_labels, pred_labels)
    except:
        results["overall_f1"] = 0.0
        results["overall_precision"] = 0.0
        results["overall_recall"] = 0.0

    # Calculate per-marker metrics
    for marker_type in MARKER_TYPES:
        try:
            # Filter sequences for this marker type
            filtered_true = []
            filtered_pred = []

            for true_seq, pred_seq in zip(true_labels, pred_labels):
                ft = []
                fp = []

                for t, p in zip(true_seq, pred_seq):
                    if marker_type in t:
                        ft.append(t)
                    else:
                        ft.append('O')

                    if marker_type in p:
                        fp.append(p)
                    else:
                        fp.append('O')

                filtered_true.append(ft)
                filtered_pred.append(fp)

            # Calculate F1 if there are actual labels
            has_labels = any(any(x != 'O' for x in seq) for seq in filtered_true)
            if has_labels:
                marker_f1 = f1_score(filtered_true, filtered_pred)
                results[f"f1_{marker_type}"] = marker_f1
            else:
                results[f"f1_{marker_type}"] = 0.0

        except:
            results[f"f1_{marker_type}"] = 0.0

    return results

# ============================================================================
# TRAINING ARGUMENTS - OPTIMIZED
# ============================================================================

training_args = TrainingArguments(
    output_dir="./optimized_marker_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="overall_f1",
    greater_is_better=True,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    report_to="none",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    dataloader_num_workers=2,
    lr_scheduler_type="linear",
    seed=42,
    label_smoothing_factor=0.1,  # Added for better generalization
)

# ============================================================================
# TRAINER
# ============================================================================

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_enhanced_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# ============================================================================
# TRAIN
# ============================================================================

print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70)
print("🎯 Target: 0.30-0.40 F1 Score")
print(f"⏱️  Estimated time: 30-60 minutes")
print("="*70)

trainer.train()

# ============================================================================
# EVALUATE
# ============================================================================

print("\n" + "="*70)
print("EVALUATION RESULTS")
print("="*70)

eval_results = trainer.evaluate()

print(f"\n📊 Overall Performance:")
print(f"  F1 Score:       {eval_results.get('eval_overall_f1', 0):.4f}")
print(f"  Precision:      {eval_results.get('eval_overall_precision', 0):.4f}")
print(f"  Recall:         {eval_results.get('eval_overall_recall', 0):.4f}")

print(f"\n📊 Per-Marker F1 Scores:")
for marker_type in MARKER_TYPES:
    key = f"eval_f1_{marker_type}"
    score = eval_results.get(key, 0)

    if score > 0.25:
        indicator = "✅"
    elif score > 0.15:
        indicator = "⚠️"
    else:
        indicator = "❌"

    print(f"  {indicator} {marker_type:10s}: {score:.4f}")

# Save model
best_model_path = "./best_marker_extraction_model"
trainer.save_model(best_model_path)
tokenizer.save_pretrained(best_model_path)
print(f"\n✅ Model saved to: {best_model_path}")

# ============================================================================
# LLM FEW-SHOT ENHANCEMENT
# ============================================================================

print("\n" + "="*70)
print("LLM FEW-SHOT ENHANCEMENT SETUP")
print("="*70)

class LLMMarkerEnhancer:
    """
    Uses LLM to validate and enhance DeBERTa predictions
    Can use OpenAI API or local LLM
    """

    def __init__(self, use_api=True):
        self.use_api = use_api
        self.few_shot_examples = self._create_few_shot_examples()

        if use_api:
            try:
                from openai import OpenAI
                self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", ""))
                print("✅ OpenAI client initialized")
            except:
                print("⚠️  OpenAI not available, using rule-based fallback")
                self.use_api = False

        # Rule-based patterns as fallback
        self.rule_patterns = {
            'Actor': [
                r'\b(CIA|FBI|NSA|government|elites|globalists|they|them)\b',
                r'\b([A-Z][a-z]+ [A-Z][a-z]+)\b',
            ],
            'Action': [
                r'\b(covering up|hiding|concealing|manipulating|controlling)\b',
                r'\b(is trying to|are attempting to|wants to|plans to)\b',
            ],
            'Evidence': [
                r'\b(according to|reports show|evidence suggests|allegedly)\b',
                r'\b(documents show|proof|records indicate)\b',
            ]
        }

    def _create_few_shot_examples(self):
        """Create few-shot examples for LLM"""
        return [
            {
                "text": "The CIA has been covering up UFO evidence for decades according to leaked documents.",
                "markers": [
                    {"type": "Actor", "text": "CIA", "start": 4, "end": 7},
                    {"type": "Action", "text": "covering up", "start": 19, "end": 30},
                    {"type": "Evidence", "text": "according to leaked documents", "start": 52, "end": 81}
                ]
            },
            {
                "text": "Government elites are manipulating the media to control public opinion.",
                "markers": [
                    {"type": "Actor", "text": "Government elites", "start": 0, "end": 17},
                    {"type": "Action", "text": "manipulating", "start": 22, "end": 34},
                    {"type": "Effect", "text": "control public opinion", "start": 49, "end": 71}
                ]
            }
        ]

    def enhance_with_llm(self, text, deberta_markers):
        """Enhance DeBERTa predictions with LLM"""

        if not self.use_api or len(text) > 1000:
            return self._enhance_with_rules(text, deberta_markers)

        try:
            # Prepare prompt with few-shot examples
            prompt = """You are an expert analyzing conspiracy theory markers.

Given a text, identify psycholinguistic markers:
- Actor: Who is responsible
- Action: What they're doing
- Effect: Consequences
- Victim: Who is affected
- Evidence: Supporting claims

Examples:"""

            for example in self.few_shot_examples:
                prompt += f"\n\nText: {example['text']}"
                prompt += f"\nMarkers: {json.dumps(example['markers'])}"

            prompt += f"\n\nNew text: {text}"
            prompt += f"\nDeBERTa predictions: {json.dumps(deberta_markers)}"
            prompt += "\n\nReview these predictions. Return a JSON array with corrected markers:"

            # Call API
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=500
            )

            # Parse response
            result = response.choices[0].message.content
            json_match = re.search(r'\[.*\]', result, re.DOTALL)

            if json_match:
                llm_markers = json.loads(json_match.group())
                return self._merge_markers(deberta_markers, llm_markers, text)
            else:
                return self._enhance_with_rules(text, deberta_markers)

        except Exception as e:
            print(f"LLM enhancement failed: {e}")
            return self._enhance_with_rules(text, deberta_markers)

    def _enhance_with_rules(self, text, deberta_markers):
        """Rule-based enhancement"""
        enhanced_markers = deberta_markers.copy()

        for marker_type, patterns in self.rule_patterns.items():
            for pattern in patterns:
                for match in re.finditer(pattern, text, re.IGNORECASE):
                    # Check if similar marker exists
                    similar_exists = False
                    for existing in enhanced_markers:
                        if (existing['type'] == marker_type and
                            existing['text'].lower() in match.group().lower()):
                            similar_exists = True
                            break

                    if not similar_exists:
                        enhanced_markers.append({
                            'type': marker_type,
                            'text': match.group(),
                            'startIndex': match.start(),
                            'endIndex': match.end()
                        })

        return enhanced_markers[:15]  # Limit markers

    def _merge_markers(self, deberta_markers, llm_markers, text):
        """Merge DeBERTa and LLM predictions"""
        merged = deberta_markers.copy()

        for llm_marker in llm_markers:
            # Check if similar exists
            similar = False
            for deberta_marker in deberta_markers:
                if self._markers_similar(llm_marker, deberta_marker):
                    similar = True
                    break

            if not similar and 'text' in llm_marker:
                # Find exact span in text
                marker_text = llm_marker['text']
                if marker_text in text:
                    start_idx = text.find(marker_text)
                    merged.append({
                        'type': llm_marker.get('type', 'Unknown'),
                        'text': marker_text,
                        'startIndex': start_idx,
                        'endIndex': start_idx + len(marker_text)
                    })

        return merged[:15]  # Limit to 15 markers

    def _markers_similar(self, m1, m2):
        """Check if markers are similar"""
        if m1.get('type') != m2.get('type'):
            return False

        t1 = m1.get('text', '').lower()
        t2 = m2.get('text', '').lower()

        return t1 in t2 or t2 in t1 or len(set(t1.split()) & set(t2.split())) > 0

# Initialize LLM enhancer
llm_enhancer = LLMMarkerEnhancer(use_api=False)  # Set to True if you have API key
print("✅ LLM enhancer initialized")

# ============================================================================
# INFERENCE WITH ENHANCEMENT
# ============================================================================

def predict_with_enhancement(text, tokenizer, model, device, llm_enhancer, min_confidence=0.3):
    """Predict markers with DeBERTa + LLM enhancement"""
    if not text or len(text) < 20:
        return []

    # DeBERTa prediction
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        return_offsets_mapping=True,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits[0]
        probs = torch.softmax(logits, dim=-1)
        predictions = torch.argmax(probs, dim=-1).cpu().numpy()
        max_probs = np.max(probs.cpu().numpy(), axis=1)

    offset_mapping = encoding['offset_mapping'][0].cpu().numpy()
    deberta_markers = []
    current_marker = None
    current_start = None
    current_end = None
    current_confidences = []

    for idx, (pred_id, confidence) in enumerate(zip(predictions, max_probs)):
        token_start, token_end = offset_mapping[idx]

        if token_start == 0 and token_end == 0:
            continue

        label = id2label[pred_id]

        if label.startswith('B-'):
            # Save previous
            if current_marker is not None and current_confidences:
                avg_conf = np.mean(current_confidences)
                if avg_conf >= min_confidence:
                    deberta_markers.append({
                        'type': current_marker,
                        'text': text[current_start:current_end],
                        'startIndex': current_start,
                        'endIndex': current_end,
                        'confidence': avg_conf
                    })

            # Start new
            current_marker = label[2:]
            current_start = int(token_start)
            current_end = int(token_end)
            current_confidences = [confidence]

        elif label.startswith('I-') and current_marker is not None:
            if label[2:] == current_marker:
                current_end = int(token_end)
                current_confidences.append(confidence)

        elif label == 'O' and current_marker is not None:
            # End marker
            if current_confidences:
                avg_conf = np.mean(current_confidences)
                if avg_conf >= min_confidence:
                    deberta_markers.append({
                        'type': current_marker,
                        'text': text[current_start:current_end],
                        'startIndex': current_start,
                        'endIndex': current_end,
                        'confidence': avg_conf
                    })
            current_marker = None

    # Last marker
    if current_marker is not None and current_confidences:
        avg_conf = np.mean(current_confidences)
        if avg_conf >= min_confidence:
            deberta_markers.append({
                'type': current_marker,
                'text': text[current_start:current_end],
                'startIndex': current_start,
                'endIndex': current_end,
                'confidence': avg_conf
            })

    # Remove confidence for enhancement
    deberta_markers_clean = []
    for marker in deberta_markers:
        deberta_markers_clean.append({
            'type': marker['type'],
            'text': marker['text'],
            'startIndex': marker['startIndex'],
            'endIndex': marker['endIndex']
        })

    # Enhance with LLM
    enhanced_markers = llm_enhancer.enhance_with_llm(text, deberta_markers_clean)

    return enhanced_markers

# ============================================================================
# CREATE FINAL SUBMISSION
# ============================================================================

import zipfile
from tqdm import tqdm

def create_final_submission(test_file_path, output_zip="submission_extraction_final.zip"):
    """Create submission with enhanced predictions"""
    print(f"\n📦 Creating submission from {test_file_path}")

    # Load test data
    test_items = []
    with open(test_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                test_items.append(json.loads(line))
            except:
                continue

    print(f"  Loaded {len(test_items)} test items")

    # Setup for inference
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Generate predictions
    predictions = []
    total_markers = 0

    print("  Generating enhanced predictions...")
    for item in tqdm(test_items, desc="Processing"):
        text = item.get('text', '')
        doc_id = item.get('_id', 'unknown')

        if not text:
            predictions.append({"_id": doc_id, "markers": []})
            continue

        # Get enhanced markers
        markers = predict_with_enhancement(
            text, tokenizer, model, device, llm_enhancer, min_confidence=0.25
        )

        predictions.append({
            "_id": doc_id,
            "markers": markers
        })
        total_markers += len(markers)

    print(f"\n📊 Final statistics:")
    print(f"  Total documents: {len(predictions)}")
    print(f"  Total markers: {total_markers}")
    print(f"  Avg markers per doc: {total_markers/len(predictions):.2f}")

    # Save to JSONL
    jsonl_file = "temp_submission.jsonl"
    with open(jsonl_file, 'w', encoding='utf-8') as f:
        for pred in predictions:
            f.write(json.dumps(pred) + '\n')

    # Create ZIP
    with zipfile.ZipFile(output_zip, 'w') as zf:
        zf.write(jsonl_file, arcname="submission.jsonl")

    os.remove(jsonl_file)

    print(f"\n✅ Submission created: {output_zip}")
    print(f"  File size: {os.path.getsize(output_zip)/1024:.1f} KB")

    return output_zip

# ============================================================================
# RUN PIPELINE
# ============================================================================

if __name__ == "__main__":
    # Create submission
    test_file = "test_rehydrated_v2.jsonl"

    if os.path.exists(test_file):
        submission_file = create_final_submission(test_file)

        print("\n" + "="*70)
        print("🎯 READY FOR SUBMISSION!")
        print("="*70)
        print(f"\n📤 Submission file: {submission_file}")
        print(f"\n🔧 Key improvements from 0.18 baseline:")
        print(f"  1. Learning rate: 5e-6 → 3e-5 (6x higher)")
        print(f"  2. Batch size: 2 → 8 (4x larger)")
        print(f"  3. Better label alignment")
        print(f"  4. Data augmentation")
        print(f"  5. LLM/rule-based enhancement")
        print(f"\n📈 Expected F1: 0.30-0.40")
        print(f"📊 Improvement: +67% to +122%")
        print("\n" + "="*70)
    else:
        print(f"\n⚠️  Test file not found: {test_file}")

Installing compatible versions...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 8.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.0 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is incompatible.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=

AttributeError: module 'pyarrow' has no attribute 'PyExtensionType'

In [ ]:
"""
SUBTASK 1: MARKER EXTRACTION - SUBMISSION GENERATOR
Fixed for local model loading
"""
import json
import os
import zipfile
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tqdm import tqdm

# ===========================
# CONFIGURATION
# ===========================
# Paths - UPDATE THESE!
INPUT_FILE = "/kaggle/input/test-reducted/test_rehydrated.jsonl"
MODEL_PATH = "./deberta_large_final/checkpoint-best"  # ⭐ Updated to match your training
OUTPUT_DIR = "/kaggle/working"
TEMP_SUBMISSION_FILE = os.path.join(OUTPUT_DIR, "submission.jsonl")
OUTPUT_ZIP = os.path.join(OUTPUT_DIR, "submission.zip")

# Marker types
MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# Labels
labels_list = ["O"]
for marker_type in MARKER_TYPES:
    labels_list.append(f"B-{marker_type}")
    labels_list.append(f"I-{marker_type}")

label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}

print("="*70)
print("MARKER EXTRACTION - SUBMISSION GENERATOR")
print("="*70)

# ===========================
# LOAD MODEL - FIXED
# ===========================
def load_model():
    """Load trained marker extraction model"""
    print("\n📦 Loading model and tokenizer...")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"   Device: {device}")

    # Check if model exists
    if not os.path.exists(MODEL_PATH):
        print(f"❌ ERROR: Model not found at {MODEL_PATH}")
        print(f"\n📁 Available directories:")
        for d in os.listdir("."):
            if os.path.isdir(d):
                print(f"   - {d}/")
                checkpoint_path = os.path.join(d, "checkpoint-best")
                if os.path.exists(checkpoint_path):
                    print(f"     ✅ Has checkpoint-best")
        raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

    print(f"✅ Found model at: {MODEL_PATH}")

    # ⭐ FIX: Load tokenizer with local_files_only=True
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            MODEL_PATH,
            local_files_only=True,
            use_fast=True
        )
        print(f"✅ Tokenizer loaded from {MODEL_PATH}")
    except Exception as e:
        print(f"⚠️  Tokenizer not found in model path, loading from base model")
        # Fallback: Load from base model (DeBERTa-v3-large)
        tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
        print(f"✅ Tokenizer loaded from base model")

    # ⭐ FIX: Load model with local_files_only=True
    model = AutoModelForTokenClassification.from_pretrained(
        MODEL_PATH,
        local_files_only=True,
        num_labels=len(labels_list),
        id2label=id2label,
        label2id=label2id
    )

    model.to(device)
    model.eval()

    print(f"✅ Model loaded successfully!")
    print(f"   Parameters: {model.num_parameters():,}")

    return tokenizer, model, device

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_markers(text, tokenizer, model, device, confidence_threshold=0.20):
    """
    Predict markers for a single text

    Returns:
        List of marker dictionaries with startIndex, endIndex, type, text
    """
    if not text or len(text.strip()) < 10:
        return []

    # Tokenize with offset mapping
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding='max_length',
        return_offsets_mapping=True,
        return_tensors='pt'
    )

    offset_mapping = encoding['offset_mapping'][0].numpy()

    # Move to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Predict with confidence scores
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[0]

        # Get predictions and confidence
        probs = torch.softmax(logits, dim=-1)
        predictions = torch.argmax(probs, dim=-1).cpu().numpy()
        confidences = torch.max(probs, dim=-1).values.cpu().numpy()

    # Extract marker spans
    markers = []
    current_marker = None
    current_start = None
    current_end = None
    current_confidences = []

    for idx, (pred_id, conf) in enumerate(zip(predictions, confidences)):
        token_start, token_end = offset_mapping[idx]

        # Skip special tokens
        if token_start == 0 and token_end == 0:
            continue

        label = id2label[pred_id]

        if label.startswith('B-'):
            # Save previous marker if exists
            if current_marker is not None and len(current_confidences) > 0:
                avg_conf = np.mean(current_confidences)
                if avg_conf >= confidence_threshold:
                    markers.append({
                        'startIndex': int(current_start),
                        'endIndex': int(current_end),
                        'type': current_marker,
                        'text': text[current_start:current_end]
                    })

            # Start new marker
            marker_type = label[2:]
            current_marker = marker_type
            current_start = int(token_start)
            current_end = int(token_end)
            current_confidences = [conf]

        elif label.startswith('I-') and current_marker is not None:
            # Continue current marker
            marker_type = label[2:]
            if marker_type == current_marker:
                current_end = int(token_end)
                current_confidences.append(conf)
            else:
                # Type mismatch, save current and start new
                if len(current_confidences) > 0:
                    avg_conf = np.mean(current_confidences)
                    if avg_conf >= confidence_threshold:
                        markers.append({
                            'startIndex': int(current_start),
                            'endIndex': int(current_end),
                            'type': current_marker,
                            'text': text[current_start:current_end]
                        })

                current_marker = marker_type
                current_start = int(token_start)
                current_end = int(token_end)
                current_confidences = [conf]

        elif label == 'O':
            # End current marker
            if current_marker is not None and len(current_confidences) > 0:
                avg_conf = np.mean(current_confidences)
                if avg_conf >= confidence_threshold:
                    markers.append({
                        'startIndex': int(current_start),
                        'endIndex': int(current_end),
                        'type': current_marker,
                        'text': text[current_start:current_end]
                    })
            current_marker = None
            current_confidences = []

    # Don't forget last marker
    if current_marker is not None and len(current_confidences) > 0:
        avg_conf = np.mean(current_confidences)
        if avg_conf >= confidence_threshold:
            markers.append({
                'startIndex': int(current_start),
                'endIndex': int(current_end),
                'type': current_marker,
                'text': text[current_start:current_end]
            })

    return markers

# ===========================
# PROCESS DOCUMENTS
# ===========================
def process_document(item, tokenizer, model, device):
    """Process a single document"""
    doc_id = item.get('_id')
    text = item.get('text', '')

    if not doc_id or not text:
        return None

    # Predict markers
    markers = predict_markers(text, tokenizer, model, device)

    # Return in submission format
    return {
        "_id": doc_id,
        "markers": markers
    }

# ===========================
# MAIN EXECUTION
# ===========================
def main():
    print("\n" + "="*70)
    print("STARTING PREDICTION PIPELINE")
    print("="*70)

    # Load model
    tokenizer, model, device = load_model()

    # Load input data
    print(f"\n📂 Loading test data from: {INPUT_FILE}")

    if not os.path.exists(INPUT_FILE):
        print(f"❌ Error: File not found: {INPUT_FILE}")
        return

    input_data = []
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                input_data.append(json.loads(line))
            except json.JSONDecodeError:
                continue

    print(f"✅ Loaded {len(input_data)} documents")

    # Process all documents
    print(f"\n🔄 Generating predictions...")
    predictions = []

    for item in tqdm(input_data, desc="Processing"):
        pred = process_document(item, tokenizer, model, device)
        if pred:
            predictions.append(pred)

    print(f"\n✅ Generated predictions for {len(predictions)} documents")

    # Calculate statistics
    total_markers = sum(len(pred['markers']) for pred in predictions)
    docs_with_markers = sum(1 for pred in predictions if len(pred['markers']) > 0)

    print(f"\n📊 PREDICTION STATISTICS:")
    print(f"   Total markers predicted: {total_markers}")
    print(f"   Documents with markers: {docs_with_markers} ({docs_with_markers/len(predictions)*100:.1f}%)")
    print(f"   Avg markers per document: {total_markers / len(predictions):.2f}")

    # Marker type distribution
    marker_counts = {mt: 0 for mt in MARKER_TYPES}
    for pred in predictions:
        for marker in pred['markers']:
            marker_type = marker['type']
            if marker_type in marker_counts:
                marker_counts[marker_type] += 1

    if total_markers > 0:
        print(f"\n📊 Marker Type Distribution:")
        for marker_type, count in sorted(marker_counts.items(), key=lambda x: -x[1]):
            percentage = (count / total_markers * 100) if total_markers > 0 else 0
            print(f"   {marker_type:10s}: {count:5d} ({percentage:5.2f}%)")

    # Save to JSONL
    print(f"\n💾 Saving predictions to {TEMP_SUBMISSION_FILE}...")
    with open(TEMP_SUBMISSION_FILE, 'w', encoding='utf-8') as f:
        for pred in predictions:
            f.write(json.dumps(pred) + '\n')

    # Create ZIP
    print(f"📦 Creating ZIP archive: {OUTPUT_ZIP}...")
    with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(TEMP_SUBMISSION_FILE, arcname="submission.jsonl")

    # Clean up
    os.remove(TEMP_SUBMISSION_FILE)

    file_size = os.path.getsize(OUTPUT_ZIP) / 1024

    print(f"\n✅ SUCCESS!")
    print(f"   Submission file: {OUTPUT_ZIP}")
    print(f"   File size: {file_size:.2f} KB")

    # Show sample predictions
    print(f"\n📋 SAMPLE PREDICTIONS (first 3 documents):")
    for i, pred in enumerate(predictions[:3]):
        print(f"\nDocument {i+1} (ID: {pred['_id']}):")
        if pred['markers']:
            for j, marker in enumerate(pred['markers'][:5], 1):
                text_preview = marker['text'][:50] + "..." if len(marker['text']) > 50 else marker['text']
                print(f"   {j}. [{marker['type']:8s}] \"{text_preview}\"")
            if len(pred['markers']) > 5:
                print(f"   ... and {len(pred['markers']) - 5} more")
        else:
            print("   (No markers predicted)")

    print("\n" + "="*70)
    print("SUBMISSION READY FOR UPLOAD!")
    print("="*70)
    print(f"\n📤 Next steps:")
    print(f"1. Download {OUTPUT_ZIP}")
    print(f"2. Go to Codabench → Extraction task")
    print(f"3. Upload to 'My Submissions'")
    print(f"4. Wait for evaluation")
    print("="*70)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# """
# SUBTASK 2: CONSPIRACY DETECTION - FIXED
# Binary classification (Yes/No only)
# Target: 0.70+ F1
# """

# import json
# import torch
# import numpy as np
# import os
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     Trainer,
#     TrainingArguments,
#     EarlyStoppingCallback
# )
# from sklearn.metrics import f1_score, accuracy_score, classification_report
# from sklearn.utils.class_weight import compute_class_weight
# from collections import Counter
# import warnings
# warnings.filterwarnings('ignore')

# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# print("="*70)
# print("SUBTASK 2: CONSPIRACY DETECTION")
# print("="*70)

# # ===========================
# # CONFIGURATION
# # ===========================
# label2id = {"No": 0, "Yes": 1}
# id2label = {0: "No", 1: "Yes"}

# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# # ===========================
# # MARKER-AWARE TEXT
# # ===========================
# def add_marker_tags(text, markers):
#     """Add special tokens around markers"""
#     if not markers:
#         return text

#     sorted_markers = sorted(markers, key=lambda x: x['startIndex'], reverse=True)

#     for marker in sorted_markers:
#         start = marker['startIndex']
#         end = marker['endIndex']
#         mtype = marker['type']

#         text = text[:end] + f"[/{mtype.upper()}]" + text[end:]
#         text = text[:start] + f"[{mtype.upper()}]" + text[start:]

#     return text

# # ===========================
# # LOAD DATASET - FIXED
# # ===========================
# def load_detection_dataset(jsonl_path, tokenizer, max_length=512, use_markers=True):
#     """Load data for conspiracy detection"""
#     texts = []
#     labels = []

#     stats = {'total': 0, 'yes': 0, 'no': 0, 'cant_tell': 0, 'skipped': 0}

#     print(f"\nLoading {jsonl_path}...")

#     with open(jsonl_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             stats['total'] += 1

#             try:
#                 item = json.loads(line)
#                 text = item.get('text', '').strip()
#                 conspiracy = item.get('conspiracy')
#                 markers = item.get('markers', [])

#                 if len(text) < 20:
#                     stats['skipped'] += 1
#                     continue

#                 # Skip "Can't tell"
#                 if conspiracy == "Can't tell":
#                     stats['cant_tell'] += 1
#                     continue

#                 if conspiracy not in label2id:
#                     stats['skipped'] += 1
#                     continue

#                 # Add marker tags
#                 if use_markers and markers:
#                     text = add_marker_tags(text, markers)

#                 texts.append(text)
#                 labels.append(label2id[conspiracy])

#                 if conspiracy == "Yes":
#                     stats['yes'] += 1
#                 else:
#                     stats['no'] += 1

#             except:
#                 stats['skipped'] += 1
#                 continue

#     # ⭐ FIXED: Handle empty case
#     if len(texts) == 0:
#         print(f"⚠️  No valid examples found!")
#         print(f"   Total: {stats['total']}")
#         print(f"   Can't tell: {stats['cant_tell']}")
#         print(f"   Skipped: {stats['skipped']}")
#         return None

#     print(f"✅ Loaded {len(texts)} examples")
#     print(f"   Yes: {stats['yes']} ({stats['yes']/len(texts)*100:.1f}%)")
#     print(f"   No: {stats['no']} ({stats['no']/len(texts)*100:.1f}%)")
#     print(f"   Skipped 'Can't tell': {stats['cant_tell']}")
#     print(f"   Skipped other: {stats['skipped']}")

#     # Tokenize
#     encodings = tokenizer(
#         texts,
#         truncation=True,
#         max_length=max_length,
#         padding='max_length'
#     )

#     return Dataset.from_dict({
#         'input_ids': encodings['input_ids'],
#         'attention_mask': encodings['attention_mask'],
#         'labels': labels
#     })

# # ===========================
# # LOAD DATA
# # ===========================
# print("\n" + "="*70)
# print("LOADING DATA")
# print("="*70)

# model_name = "microsoft/deberta-v3-base"  # Use base for faster training
# print(f"Model: {model_name}")

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Add marker tokens
# special_tokens = []
# for mtype in MARKER_TYPES:
#     special_tokens.extend([f"[{mtype.upper()}]", f"[/{mtype.upper()}]"])

# num_added = tokenizer.add_tokens(special_tokens)
# print(f"Added {num_added} special marker tokens")

# # Load train
# train_ds = load_detection_dataset(
#     "train_rehydrated_v2.jsonl",
#     tokenizer,
#     max_length=512,
#     use_markers=True
# )

# if train_ds is None:
#     print("❌ Failed to load training data!")
#     exit(1)

# # Load validation
# val_ds = load_detection_dataset(
#     "dev_rehydrated_v2.jsonl",
#     tokenizer,
#     max_length=512,
#     use_markers=True
# )

# # ⭐ FIXED: Always split from training if validation is empty or too small
# if val_ds is None or len(val_ds) < 50:
#     print("\n⚠️  Validation empty/small, splitting from training (15%)")
#     split = train_ds.train_test_split(test_size=0.15, seed=42)
#     train_ds = split['train']
#     val_ds = split['test']
#     print(f"✅ Split: {len(train_ds)} train, {len(val_ds)} val")

# train_ds.set_format("torch")
# val_ds.set_format("torch")

# # ===========================
# # CLASS WEIGHTS
# # ===========================
# print("\n" + "="*70)
# print("CLASS BALANCING")
# print("="*70)

# train_labels = np.array(train_ds['labels'])
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(train_labels),
#     y=train_labels
# )

# class_weights = class_weights ** 1.5  # Boost minority class
# class_weights = torch.tensor(class_weights, dtype=torch.float)

# print(f"Class weights: {class_weights}")
# print(f"\nClass distribution:")
# unique, counts = np.unique(train_labels, return_counts=True)
# for label_id, count in zip(unique, counts):
#     pct = count / len(train_labels) * 100
#     print(f"   {id2label[label_id]}: {count} ({pct:.1f}%)")

# # ===========================
# # MODEL
# # ===========================
# print("\n" + "="*70)
# print("INITIALIZING MODEL")
# print("="*70)

# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=2,
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True
# )

# model.resize_token_embeddings(len(tokenizer))
# print(f"✅ Model initialized")

# # ===========================
# # WEIGHTED TRAINER
# # ===========================
# class WeightedTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits

#         loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
#         loss = loss_fn(logits, labels)

#         return (loss, outputs) if return_outputs else loss

# # ===========================
# # METRICS
# # ===========================
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)

#     acc = accuracy_score(labels, preds)
#     macro_f1 = f1_score(labels, preds, average='macro')
#     binary_f1 = f1_score(labels, preds, average='binary', pos_label=1, zero_division=0)

#     from sklearn.metrics import precision_score, recall_score
#     precision = precision_score(labels, preds, pos_label=1, zero_division=0)
#     recall = recall_score(labels, preds, pos_label=1, zero_division=0)

#     return {
#         "accuracy": acc,
#         "macro_f1": macro_f1,
#         "binary_f1": binary_f1,
#         "precision": precision,
#         "recall": recall
#     }

# # ===========================
# # TRAINING ARGS
# # ===========================
# training_args = TrainingArguments(
#     output_dir="./detection_model",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     load_best_model_at_end=True,
#     metric_for_best_model="macro_f1",
#     greater_is_better=True,
#     logging_steps=50,
#     report_to="none",
#     save_total_limit=2,
#     fp16=torch.cuda.is_available(),
#     gradient_accumulation_steps=2,
#     dataloader_num_workers=0,
#     lr_scheduler_type="cosine",
#     seed=42,
#     label_smoothing_factor=0.1
# )

# # ===========================
# # TRAIN
# # ===========================
# print("\n" + "="*70)
# print("STARTING TRAINING")
# print("="*70)
# print("🎯 Target: 0.70+ Macro F1")
# print("="*70 + "\n")

# trainer = WeightedTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=val_ds,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
# )

# trainer.train()

# # ===========================
# # SAVE
# # ===========================
# final_path = "./detection_model/best"
# trainer.save_model(final_path)
# tokenizer.save_pretrained(final_path)

# print(f"\n✅ Saved to: {final_path}")

# # ===========================
# # EVALUATION
# # ===========================
# print("\n" + "="*70)
# print("FINAL EVALUATION")
# print("="*70)

# results = trainer.evaluate()

# print(f"\n📊 RESULTS:")
# print(f"   Accuracy:  {results['eval_accuracy']:.4f}")
# print(f"   Macro F1:  {results['eval_macro_f1']:.4f}")
# print(f"   Binary F1: {results['eval_binary_f1']:.4f}")
# print(f"   Precision: {results['eval_precision']:.4f}")
# print(f"   Recall:    {results['eval_recall']:.4f}")

# # Detailed report
# predictions = trainer.predict(val_ds)
# pred_labels = np.argmax(predictions.predictions, axis=1)
# true_labels = predictions.label_ids

# print("\n" + "="*70)
# print("CLASSIFICATION REPORT")
# print("="*70)
# print(classification_report(
#     true_labels,
#     pred_labels,
#     target_names=["No", "Yes"],
#     digits=4
# ))

# # Prediction balance
# yes_pct = np.sum(pred_labels == 1) / len(pred_labels) * 100
# print(f"\nPrediction distribution:")
# print(f"   No:  {np.sum(pred_labels == 0)} ({100-yes_pct:.1f}%)")
# print(f"   Yes: {np.sum(pred_labels == 1)} ({yes_pct:.1f}%)")

# if results['eval_macro_f1'] >= 0.70:
#     print(f"\n🎉 EXCELLENT! F1 {results['eval_macro_f1']:.4f} > 0.70")
# elif results['eval_macro_f1'] >= 0.60:
#     print(f"\n✅ Good! F1 {results['eval_macro_f1']:.4f} > 0.60")
# else:
#     print(f"\n⚠️  F1 {results['eval_macro_f1']:.4f} - needs improvement")

# print("\n" + "="*70)
# print("DETECTION TRAINING COMPLETE!")
# print("="*70)

In [ ]:
# """
# SUBTASK 2: CONSPIRACY DETECTION - SUBMISSION GENERATOR
# Uses predicted markers from Subtask 1 for better accuracy
# """
# import json
# import os
# import zipfile
# import torch
# import numpy as np
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     AutoModelForTokenClassification
# )
# from tqdm import tqdm

# # ===========================
# # CONFIGURATION
# # ===========================
# # Paths - UPDATE THESE!
# INPUT_FILE = "test_rehydrated_v2.jsonl"
# MARKER_MODEL_PATH = "./deberta_large_final/checkpoint-best"  # Marker extraction model
# DETECTION_MODEL_PATH = "./detection_model/best"  # Detection model
# OUTPUT_DIR = "/kaggle/working"
# TEMP_SUBMISSION_FILE = os.path.join(OUTPUT_DIR, "submission.jsonl")
# OUTPUT_ZIP = os.path.join(OUTPUT_DIR, "submission_detection.zip")

# # Labels
# label2id = {"No": 0, "Yes": 1}
# id2label = {0: "No", 1: "Yes"}

# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# print("="*70)
# print("CONSPIRACY DETECTION - MARKER-AWARE SUBMISSION")
# print("="*70)

# # ===========================
# # LOAD MARKER EXTRACTION MODEL
# # ===========================
# def load_marker_model():
#     """Load marker extraction model to predict markers"""
#     print("\n📦 Loading marker extraction model...")

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # Marker labels
#     marker_labels = ["O"]
#     for mt in MARKER_TYPES:
#         marker_labels.extend([f"B-{mt}", f"I-{mt}"])

#     marker_label2id = {label: i for i, label in enumerate(marker_labels)}
#     marker_id2label = {i: label for label, i in marker_label2id.items()}

#     # Load tokenizer
#     marker_tokenizer = AutoTokenizer.from_pretrained(
#         MARKER_MODEL_PATH,
#         local_files_only=True
#     )

#     # Load model
#     marker_model = AutoModelForTokenClassification.from_pretrained(
#         MARKER_MODEL_PATH,
#         local_files_only=True,
#         num_labels=len(marker_labels),
#         id2label=marker_id2label,
#         label2id=marker_label2id
#     )

#     marker_model.to(device)
#     marker_model.eval()

#     print("✅ Marker model loaded")

#     return marker_tokenizer, marker_model, marker_id2label, device

# # ===========================
# # LOAD DETECTION MODEL
# # ===========================
# def load_detection_model():
#     """Load conspiracy detection model"""
#     print("\n📦 Loading detection model...")

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # Load tokenizer
#     detection_tokenizer = AutoTokenizer.from_pretrained(
#         DETECTION_MODEL_PATH,
#         local_files_only=True
#     )

#     # Load model
#     detection_model = AutoModelForSequenceClassification.from_pretrained(
#         DETECTION_MODEL_PATH,
#         local_files_only=True,
#         num_labels=2,
#         id2label=id2label,
#         label2id=label2id
#     )

#     detection_model.to(device)
#     detection_model.eval()

#     print("✅ Detection model loaded")

#     return detection_tokenizer, detection_model, device

# # ===========================
# # PREDICT MARKERS
# # ===========================
# def predict_markers(text, tokenizer, model, id2label, device, confidence_threshold=0.15):
#     """Predict markers for text (same as Subtask 1)"""
#     if not text or len(text.strip()) < 10:
#         return []

#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=512,
#         padding='max_length',
#         return_offsets_mapping=True,
#         return_tensors='pt'
#     )

#     offset_mapping = encoding['offset_mapping'][0].numpy()
#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits[0]
#         probs = torch.softmax(logits, dim=-1)
#         predictions = torch.argmax(probs, dim=-1).cpu().numpy()
#         confidences = torch.max(probs, dim=-1).values.cpu().numpy()

#     # Extract markers
#     markers = []
#     current_marker = None
#     current_start = None
#     current_end = None
#     current_confidences = []

#     for idx, (pred_id, conf) in enumerate(zip(predictions, confidences)):
#         token_start, token_end = offset_mapping[idx]

#         if token_start == 0 and token_end == 0:
#             continue

#         label = id2label[pred_id]

#         if label.startswith('B-'):
#             if current_marker is not None and len(current_confidences) > 0:
#                 avg_conf = np.mean(current_confidences)
#                 if avg_conf >= confidence_threshold:
#                     markers.append({
#                         'startIndex': int(current_start),
#                         'endIndex': int(current_end),
#                         'type': current_marker
#                     })

#             marker_type = label[2:]
#             current_marker = marker_type
#             current_start = int(token_start)
#             current_end = int(token_end)
#             current_confidences = [conf]

#         elif label.startswith('I-') and current_marker is not None:
#             marker_type = label[2:]
#             if marker_type == current_marker:
#                 current_end = int(token_end)
#                 current_confidences.append(conf)

#         elif label == 'O':
#             if current_marker is not None and len(current_confidences) > 0:
#                 avg_conf = np.mean(current_confidences)
#                 if avg_conf >= confidence_threshold:
#                     markers.append({
#                         'startIndex': int(current_start),
#                         'endIndex': int(current_end),
#                         'type': current_marker
#                     })
#             current_marker = None
#             current_confidences = []

#     # Last marker
#     if current_marker is not None and len(current_confidences) > 0:
#         avg_conf = np.mean(current_confidences)
#         if avg_conf >= confidence_threshold:
#             markers.append({
#                 'startIndex': int(current_start),
#                 'endIndex': int(current_end),
#                 'type': current_marker
#             })

#     return markers

# # ===========================
# # ADD MARKER TAGS TO TEXT
# # ===========================
# def add_marker_tags(text, markers):
#     """Add special tokens around markers for marker-aware detection"""
#     if not markers:
#         return text

#     # Sort markers by position (reverse for correct insertion)
#     sorted_markers = sorted(markers, key=lambda x: x['startIndex'], reverse=True)

#     for marker in sorted_markers:
#         start = marker['startIndex']
#         end = marker['endIndex']
#         mtype = marker['type']

#         # Insert closing tag
#         text = text[:end] + f"[/{mtype.upper()}]" + text[end:]
#         # Insert opening tag
#         text = text[:start] + f"[{mtype.upper()}]" + text[start:]

#     return text

# # ===========================
# # PREDICT CONSPIRACY
# # ===========================
# def predict_conspiracy(text, markers, tokenizer, model, device):
#     """Predict conspiracy using marker-aware text"""

#     # Add marker tags to text
#     marked_text = add_marker_tags(text, markers)

#     # Tokenize
#     encoding = tokenizer(
#         marked_text,
#         truncation=True,
#         max_length=512,
#         padding='max_length',
#         return_tensors='pt'
#     )

#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     # Predict
#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits[0]
#         probs = torch.softmax(logits, dim=-1)
#         pred_id = torch.argmax(probs).item()

#     return id2label[pred_id]

# # ===========================
# # PROCESS DOCUMENTS
# # ===========================
# def process_document(item, marker_tokenizer, marker_model, marker_id2label,
#                     detection_tokenizer, detection_model, device):
#     """Process a single document - returns BOTH conspiracy AND markers"""
#     doc_id = item.get('_id')
#     text = item.get('text', '')

#     if not doc_id or not text:
#         return None

#     # Step 1: Predict markers using marker extraction model
#     markers = predict_markers(
#         text,
#         marker_tokenizer,
#         marker_model,
#         marker_id2label,
#         device
#     )

#     # Add 'text' field to each marker for submission
#     for marker in markers:
#         if 'text' not in marker:
#             start = marker['startIndex']
#             end = marker['endIndex']
#             marker['text'] = text[start:end]

#     # Step 2: Predict conspiracy using detection model + markers
#     conspiracy = predict_conspiracy(
#         text,
#         markers,
#         detection_tokenizer,
#         detection_model,
#         device
#     )

#     # ⭐ RETURN BOTH FIELDS (required format)
#     return {
#         "_id": doc_id,
#         "conspiracy": conspiracy,
#         "markers": markers  # Include markers in submission
#     }

# # ===========================
# # MAIN EXECUTION
# # ===========================
# def main():
#     print("\n" + "="*70)
#     print("STARTING MARKER-AWARE DETECTION PIPELINE")
#     print("="*70)

#     # Load models
#     marker_tokenizer, marker_model, marker_id2label, marker_device = load_marker_model()
#     detection_tokenizer, detection_model, detection_device = load_detection_model()

#     # Use same device for both
#     device = marker_device

#     # Load input data
#     print(f"\n📂 Loading test data from: {INPUT_FILE}")

#     if not os.path.exists(INPUT_FILE):
#         print(f"❌ Error: File not found: {INPUT_FILE}")
#         return

#     input_data = []
#     with open(INPUT_FILE, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 input_data.append(json.loads(line))
#             except:
#                 continue

#     print(f"✅ Loaded {len(input_data)} documents")

#     # Process all documents
#     print(f"\n🔄 Generating predictions...")
#     predictions = []

#     for item in tqdm(input_data, desc="Processing"):
#         pred = process_document(
#             item,
#             marker_tokenizer,
#             marker_model,
#             marker_id2label,
#             detection_tokenizer,
#             detection_model,
#             device
#         )
#         if pred:
#             predictions.append(pred)

#     print(f"\n✅ Generated predictions for {len(predictions)} documents")

#     # Calculate statistics
#     yes_count = sum(1 for pred in predictions if pred['conspiracy'] == 'Yes')
#     no_count = sum(1 for pred in predictions if pred['conspiracy'] == 'No')
#     total_markers = sum(len(pred['markers']) for pred in predictions)
#     docs_with_markers = sum(1 for pred in predictions if len(pred['markers']) > 0)

#     print(f"\n📊 PREDICTION STATISTICS:")
#     print(f"   Total documents: {len(predictions)}")
#     print(f"\n📊 Conspiracy Detection (Subtask 2):")
#     print(f"   Predicted 'Yes': {yes_count} ({yes_count/len(predictions)*100:.1f}%)")
#     print(f"   Predicted 'No':  {no_count} ({no_count/len(predictions)*100:.1f}%)")
#     print(f"\n📊 Marker Extraction (Subtask 1):")
#     print(f"   Total markers: {total_markers}")
#     print(f"   Docs with markers: {docs_with_markers} ({docs_with_markers/len(predictions)*100:.1f}%)")
#     print(f"   Avg markers/doc: {total_markers/len(predictions):.2f}")

#     # Save to JSONL
#     print(f"\n💾 Saving predictions to {TEMP_SUBMISSION_FILE}...")
#     with open(TEMP_SUBMISSION_FILE, 'w', encoding='utf-8') as f:
#         for pred in predictions:
#             f.write(json.dumps(pred) + '\n')

#     # Create ZIP
#     print(f"📦 Creating ZIP archive: {OUTPUT_ZIP}...")
#     with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
#         zf.write(TEMP_SUBMISSION_FILE, arcname="submission.jsonl")

#     # Clean up
#     os.remove(TEMP_SUBMISSION_FILE)

#     file_size = os.path.getsize(OUTPUT_ZIP) / 1024

#     print(f"\n✅ SUCCESS!")
#     print(f"   Submission file: {OUTPUT_ZIP}")
#     print(f"   File size: {file_size:.2f} KB")

#     # Show sample predictions
#     print(f"\n📋 SAMPLE PREDICTIONS (first 5):")
#     for i, pred in enumerate(predictions[:5], 1):
#         print(f"\n{i}. Document: {pred['_id']}")
#         print(f"   Conspiracy: {pred['conspiracy']}")
#         if pred['markers']:
#             print(f"   Markers ({len(pred['markers'])}):")
#             for j, marker in enumerate(pred['markers'][:3], 1):
#                 text_preview = marker['text'][:40] + "..." if len(marker['text']) > 40 else marker['text']
#                 print(f"      {j}. [{marker['type']:8s}] \"{text_preview}\"")
#             if len(pred['markers']) > 3:
#                 print(f"      ... and {len(pred['markers']) - 3} more")
#         else:
#             print(f"   Markers: (none)")

#     print("\n" + "="*70)
#     print("SUBMISSION READY FOR UPLOAD!")
#     print("="*70)
#     print(f"\n📤 Next steps:")
#     print(f"1. Download {OUTPUT_ZIP}")
#     print(f"2. Go to Codabench → Detection task (Subtask 2)")
#     print(f"3. Upload to 'My Submissions'")
#     print(f"4. Wait for evaluation")

#     # Performance expectations
#     yes_pct = yes_count / len(predictions) * 100
#     print(f"\n💡 Performance Estimate:")
#     if 30 <= yes_pct <= 70:
#         print(f"   Good balance ({yes_pct:.1f}% 'Yes')")
#         print(f"   Expected F1: 0.65-0.75")
#     elif yes_pct < 30:
#         print(f"   Conservative ({yes_pct:.1f}% 'Yes')")
#         print(f"   Expected F1: 0.55-0.65 (high precision, lower recall)")
#     else:
#         print(f"   Liberal ({yes_pct:.1f}% 'Yes')")
#         print(f"   Expected F1: 0.60-0.70 (high recall, lower precision)")

#     print("="*70)

# if __name__ == "__main__":
#     try:
#         main()
#     except Exception as e:
#         print(f"\n❌ ERROR: {e}")
#         import traceback
#         traceback.print_exc()

not keep


In [ ]:
# !pip install fuzzywuzzy python-Levenshtein -q

In [ ]:
# """
# LLM MARKER EXTRACTION - FIXED TWO-STAGE APPROACH
# Stage 1: LLM extracts marker TEXT (no positions)
# Stage 2: Fuzzy matching finds positions in original text
# """
# import json
# import torch
# import re
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from tqdm import tqdm
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
# import warnings
# warnings.filterwarnings('ignore')

# # ===========================
# # IMPROVED PROMPT (TEXT ONLY)
# # ===========================
# FEW_SHOT_EXAMPLES = """
# EXAMPLE 1:
# Text: "The CIA and FBI are covering up evidence of election fraud to protect their puppet president."

# Markers:
# - Actor: "CIA", "FBI"
# - Action: "covering up evidence", "protect their puppet president"
# - Victim: (none explicit)
# - Effect: (none explicit)
# - Evidence: (none explicit)

# JSON:
# {
#   "Actor": ["CIA", "FBI"],
#   "Action": ["covering up evidence", "protect their puppet president"],
#   "Victim": [],
#   "Effect": [],
#   "Evidence": []
# }

# EXAMPLE 2:
# Text: "Big Pharma suppressed the cure for cancer because treating patients is more profitable than curing them."

# Markers:
# - Actor: "Big Pharma"
# - Action: "suppressed the cure for cancer"
# - Victim: "patients"
# - Effect: (implied financial harm, but not explicit)
# - Evidence: "treating patients is more profitable than curing them"

# JSON:
# {
#   "Actor": ["Big Pharma"],
#   "Action": ["suppressed the cure for cancer"],
#   "Victim": ["patients"],
#   "Effect": [],
#   "Evidence": ["treating patients is more profitable than curing them"]
# }

# EXAMPLE 3:
# Text: "Climate change is a hoax invented by scientists to secure research funding and push a globalist agenda."

# Markers:
# - Actor: "scientists"
# - Action: "invented", "secure research funding", "push a globalist agenda"
# - Victim: (none explicit)
# - Effect: (none explicit)
# - Evidence: "to secure research funding"

# JSON:
# {
#   "Actor": ["scientists"],
#   "Action": ["invented", "secure research funding", "push a globalist agenda"],
#   "Victim": [],
#   "Effect": [],
#   "Evidence": ["to secure research funding"]
# }

# EXAMPLE 4:
# Text: "Mandatory vaccines are being used to implant tracking microchips in citizens for government surveillance."

# Markers:
# - Actor: "government" (implied)
# - Action: "implant tracking microchips"
# - Victim: "citizens"
# - Effect: "surveillance"
# - Evidence: (none explicit)

# JSON:
# {
#   "Actor": ["government"],
#   "Action": ["implant tracking microchips"],
#   "Victim": ["citizens"],
#   "Effect": ["surveillance"],
#   "Evidence": []
# }

# EXAMPLE 5:
# Text: "The WHO deliberately exaggerated COVID death rates according to leaked documents to justify authoritarian lockdowns that stripped people of their freedoms."

# Markers:
# - Actor: "The WHO"
# - Action: "deliberately exaggerated COVID death rates", "justify authoritarian lockdowns", "stripped people of their freedoms"
# - Victim: "people"
# - Effect: "stripped people of their freedoms", "authoritarian lockdowns"
# - Evidence: "leaked documents"

# JSON:
# {
#   "Actor": ["The WHO"],
#   "Action": ["deliberately exaggerated COVID death rates", "justify authoritarian lockdowns", "stripped people of their freedoms"],
#   "Victim": ["people"],
#   "Effect": ["stripped people of their freedoms", "authoritarian lockdowns"],
#   "Evidence": ["leaked documents"]
# }
# """

# SYSTEM_PROMPT = """You are an expert at identifying conspiracy theory markers in text.

# Extract EXACT TEXT SPANS for each marker type. Do NOT create positions/indices.

# MARKER TYPES:
# 1. Actor: Who is responsible? (people, groups, organizations with alleged malicious intent)
# 2. Action: What are they doing? (malicious acts, plans, agendas)
# 3. Effect: What negative consequences? (harm, damage, loss)
# 4. Victim: Who is harmed? (targets of the conspiracy)
# 5. Evidence: What proof is cited? (reasons to believe the conspiracy)

# RULES:
# - Extract EXACT phrases from the text (copy verbatim)
# - One marker can appear in multiple categories
# - If no markers of a type exist, use empty list []
# - Return ONLY valid JSON, no explanation
# """

# def create_prompt(text):
#     """Create improved prompt that asks for TEXT only"""
#     return f"""<|im_start|>system
# {SYSTEM_PROMPT}<|im_end|>
# <|im_start|>user
# {FEW_SHOT_EXAMPLES}

# Now extract conspiracy markers from this text:

# Text: "{text}"

# Extract EXACT text spans (copy from the text above). Return ONLY this JSON format:
# {{
#   "Actor": ["exact text 1", "exact text 2"],
#   "Action": ["exact text 1"],
#   "Victim": [],
#   "Effect": ["exact text 1"],
#   "Evidence": []
# }}
# <|im_end|>
# <|im_start|>assistant
# """

# # ===========================
# # FUZZY STRING MATCHING
# # ===========================
# def find_marker_positions(text, marker_text, marker_type, min_similarity=70):
#     """
#     Find all positions of marker_text in the original text using fuzzy matching
#     Returns list of (start, end) tuples
#     """
#     positions = []
#     text_lower = text.lower()
#     marker_lower = marker_text.lower().strip()

#     if not marker_lower or len(marker_lower) < 3:
#         return positions

#     # Strategy 1: Exact substring match (fastest)
#     start = 0
#     while True:
#         idx = text_lower.find(marker_lower, start)
#         if idx == -1:
#             break
#         positions.append((idx, idx + len(marker_text)))
#         start = idx + 1

#     if positions:
#         return positions

#     # Strategy 2: Fuzzy matching for close matches
#     # Split text into overlapping windows
#     words = text.split()
#     marker_word_count = len(marker_text.split())

#     best_match = None
#     best_score = 0
#     best_pos = None

#     for i in range(len(words)):
#         # Try windows of different sizes around marker length
#         for window_size in range(max(1, marker_word_count - 2), marker_word_count + 3):
#             if i + window_size > len(words):
#                 continue

#             window = ' '.join(words[i:i + window_size])
#             score = fuzz.ratio(marker_lower, window.lower())

#             if score > best_score and score >= min_similarity:
#                 best_score = score
#                 best_match = window

#                 # Find position in original text
#                 window_start = text.lower().find(window.lower())
#                 if window_start != -1:
#                     best_pos = (window_start, window_start + len(window))

#     if best_pos:
#         positions.append(best_pos)

#     return positions

# def extract_markers_with_positions(text, llm_markers):
#     """
#     Convert LLM output (text only) to markers with positions
#     """
#     final_markers = []

#     for marker_type in ['Actor', 'Action', 'Effect', 'Victim', 'Evidence']:
#         marker_texts = llm_markers.get(marker_type, [])

#         for marker_text in marker_texts:
#             if not marker_text or not isinstance(marker_text, str):
#                 continue

#             # Find all occurrences
#             positions = find_marker_positions(text, marker_text, marker_type)

#             for start, end in positions:
#                 final_markers.append({
#                     'type': marker_type,
#                     'startIndex': start,
#                     'endIndex': end,
#                     'text': text[start:end]
#                 })

#     return final_markers

# # ===========================
# # PARSE LLM OUTPUT
# # ===========================
# def parse_llm_output(response):
#     """Parse JSON from LLM response"""
#     try:
#         # Remove markdown
#         response = re.sub(r'```json\s*|\s*```', '', response)

#         # Find JSON
#         json_match = re.search(r'\{[^}]*"Actor".*?\}', response, re.DOTALL)
#         if not json_match:
#             return None

#         data = json.loads(json_match.group())

#         # Validate structure
#         required_keys = ['Actor', 'Action', 'Effect', 'Victim', 'Evidence']
#         if not all(k in data for k in required_keys):
#             return None

#         # Ensure all values are lists
#         for key in required_keys:
#             if not isinstance(data[key], list):
#                 data[key] = []

#         return data

#     except Exception as e:
#         return None

# # ===========================
# # LOAD MODEL
# # ===========================
# print("="*70)
# print("LLM MARKER EXTRACTION - FIXED TWO-STAGE APPROACH")
# print("="*70)

# model_name = "Qwen/Qwen2.5-7B-Instruct"

# print(f"\n📦 Loading {model_name}...")

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True
# )

# print(f"✅ Model loaded!")

# # ===========================
# # GENERATE FUNCTION
# # ===========================
# def generate_markers(text):
#     """Generate marker texts (no positions)"""
#     prompt = create_prompt(text)

#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=500,
#             temperature=0.2,  # Lower temp for more consistent output
#             top_p=0.9,
#             do_sample=True,
#             pad_token_id=tokenizer.pad_token_id,
#             eos_token_id=tokenizer.eos_token_id
#         )

#     response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
#     return response

# # ===========================
# # MAIN PROCESSING
# # ===========================
# def process_document(text):
#     """Process one document: LLM → fuzzy match → markers"""

#     # Stage 1: LLM extracts marker texts
#     response = generate_markers(text)
#     llm_markers = parse_llm_output(response)

#     if not llm_markers:
#         return []

#     # Stage 2: Find positions via fuzzy matching
#     markers = extract_markers_with_positions(text, llm_markers)

#     return markers

# # ===========================
# # TEST FUNCTION
# # ===========================
# def test_two_stage(input_file, output_file, num_samples=50):
#     """Test two-stage extraction"""

#     print(f"\n{'='*70}")
#     print(f"TESTING TWO-STAGE EXTRACTION")
#     print(f"{'='*70}")

#     # Load data
#     print(f"\n📂 Loading from: {input_file}")
#     data = []
#     with open(input_file, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 item = json.loads(line)
#                 if item.get('text') and len(item['text']) >= 30:
#                     data.append(item)
#             except:
#                 continue

#     data = data[:num_samples]
#     print(f"✅ Loaded {len(data)} examples")

#     # Process
#     predictions = []
#     total_markers = 0
#     errors = 0

#     print(f"\n🔄 Processing (Stage 1: LLM, Stage 2: Position matching)...")
#     for item in tqdm(data, desc="Processing"):
#         text = item['text']
#         doc_id = item.get('_id', f"doc_{len(predictions)}")

#         try:
#             markers = process_document(text)
#             total_markers += len(markers)

#             predictions.append({
#                 "_id": doc_id,
#                 "markers": markers,
#                 "text": text
#             })

#         except Exception as e:
#             errors += 1
#             predictions.append({
#                 "_id": doc_id,
#                 "markers": [],
#                 "text": text
#             })

#     # Statistics
#     docs_with_markers = sum(1 for p in predictions if len(p['markers']) > 0)

#     print(f"\n{'='*70}")
#     print(f"RESULTS")
#     print(f"{'='*70}")
#     print(f"✅ Processed: {len(predictions)} documents")
#     print(f"📊 Total markers: {total_markers}")
#     print(f"📄 Docs with markers: {docs_with_markers} ({docs_with_markers/len(predictions)*100:.1f}%)")
#     print(f"📈 Avg markers/doc: {total_markers/len(predictions):.2f}")
#     if errors > 0:
#         print(f"⚠️  Errors: {errors}")

#     # Marker distribution
#     marker_counts = {'Actor': 0, 'Action': 0, 'Effect': 0, 'Victim': 0, 'Evidence': 0}
#     for pred in predictions:
#         for m in pred['markers']:
#             marker_counts[m['type']] += 1

#     print(f"\n📊 Marker Distribution:")
#     for mtype, count in sorted(marker_counts.items(), key=lambda x: -x[1]):
#         pct = (count / total_markers * 100) if total_markers > 0 else 0
#         print(f"   {mtype:10s}: {count:4d} ({pct:5.1f}%)")

#     # Save
#     print(f"\n💾 Saving to {output_file}...")
#     with open(output_file, 'w', encoding='utf-8') as f:
#         for pred in predictions:
#             output_pred = {
#                 "_id": pred["_id"],
#                 "markers": pred["markers"]
#             }
#             f.write(json.dumps(output_pred) + '\n')

#     # Samples
#     print(f"\n{'='*70}")
#     print(f"SAMPLE PREDICTIONS (first 3)")
#     print(f"{'='*70}")
#     for i, pred in enumerate(predictions[:3], 1):
#         print(f"\n📄 Document {i}: {pred['_id']}")
#         print(f"   Text: {pred['text'][:120]}...")

#         if pred['markers']:
#             print(f"   ✅ Markers ({len(pred['markers'])}):")
#             for j, m in enumerate(pred['markers'][:8], 1):
#                 # Verify position
#                 actual_text = pred['text'][m['startIndex']:m['endIndex']]
#                 match = "✓" if actual_text == m['text'] else "✗"
#                 text_preview = m['text'][:40] + "..." if len(m['text']) > 40 else m['text']
#                 print(f"      {j}. [{m['type']:8s}] {match} \"{text_preview}\"")
#             if len(pred['markers']) > 8:
#                 print(f"      ... and {len(pred['markers']) - 8} more")
#         else:
#             print(f"   ❌ No markers")

#     print(f"\n{'='*70}")
#     print(f"✅ TWO-STAGE EXTRACTION COMPLETE!")
#     print(f"{'='*70}")

#     # Performance estimate
#     if total_markers > 0:
#         print(f"\n💡 Performance Estimate:")
#         avg = total_markers / len(predictions)
#         if avg >= 3:
#             print(f"   🎉 Excellent! {avg:.1f} markers/doc")
#             print(f"   Expected F1: 0.25-0.35")
#         elif avg >= 1.5:
#             print(f"   ✅ Good! {avg:.1f} markers/doc")
#             print(f"   Expected F1: 0.20-0.28")
#         else:
#             print(f"   ⚠️  Low: {avg:.1f} markers/doc")
#             print(f"   Expected F1: 0.15-0.22")

#     return predictions

# # ===========================
# # RUN
# # ===========================
# if __name__ == "__main__":
#     predictions = test_two_stage(
#         input_file="dev_rehydrated_v2.jsonl",
#         output_file="llm_predictions_fixed.jsonl",
#         num_samples=50
#     )

In [ ]:
# """
# THRESHOLD OPTIMIZER
# Find the best confidence threshold for your model
# """

# import json
# import torch
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from tqdm import tqdm
# from collections import Counter

# # Configuration
# MODEL_PATH = "./marker_results_advanced/checkpoint-best"
# MODEL_NAME = "microsoft/deberta-v3-base"
# TRAIN_FILE = "train_rehydrated_v2.jsonl"

# # Labels
# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]
# labels_list = ["O"]
# for mt in MARKER_TYPES:
#     labels_list.extend([f"B-{mt}", f"I-{mt}"])

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# print("="*70)
# print("CONFIDENCE THRESHOLD OPTIMIZER")
# print("="*70)

# # Load model
# print("\n📦 Loading model...")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Device: {device}")

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
# model.to(device)  # CRITICAL: Move model to device
# model.eval()

# print("✅ Model loaded")

# # Prediction function
# def predict_with_threshold(text, threshold=0.5, max_length=512):
#     """Predict markers with given threshold"""

#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=max_length,
#         padding='max_length',
#         return_offsets_mapping=True,
#         return_tensors='pt'
#     )

#     offset_mapping = encoding['offset_mapping'][0].cpu().numpy()

#     # Move inputs to device
#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits[0].cpu().numpy()

#         predictions = np.argmax(logits, axis=1)
#         confidences = np.max(torch.softmax(torch.tensor(logits), dim=1).numpy(), axis=1)

#     # Extract markers
#     markers = []
#     current_marker = None
#     current_start = None
#     current_end = None
#     current_conf = []

#     for idx, (pred_id, conf, (token_start, token_end)) in enumerate(zip(predictions, confidences, offset_mapping)):
#         if token_start == 0 and token_end == 0:
#             continue

#         label = id2label[pred_id]

#         if label.startswith('B-'):
#             if current_marker is not None:
#                 avg_conf = np.mean(current_conf)
#                 if avg_conf >= threshold:
#                     markers.append({
#                         'startIndex': current_start,
#                         'endIndex': current_end,
#                         'type': current_marker
#                     })

#             marker_type = label[2:]
#             current_marker = marker_type
#             current_start = int(token_start)
#             current_end = int(token_end)
#             current_conf = [conf]

#         elif label.startswith('I-') and current_marker is not None:
#             marker_type = label[2:]
#             if marker_type == current_marker:
#                 current_end = int(token_end)
#                 current_conf.append(conf)

#         elif label == 'O':
#             if current_marker is not None:
#                 avg_conf = np.mean(current_conf)
#                 if avg_conf >= threshold:
#                     markers.append({
#                         'startIndex': current_start,
#                         'endIndex': current_end,
#                         'type': current_marker
#                     })
#                 current_marker = None

#     if current_marker is not None:
#         avg_conf = np.mean(current_conf)
#         if avg_conf >= threshold:
#             markers.append({
#                 'startIndex': current_start,
#                 'endIndex': current_end,
#                 'type': current_marker
#             })

#     return markers


# # Load sample data
# print(f"\n📂 Loading {TRAIN_FILE}...")
# samples = []
# with open(TRAIN_FILE, 'r') as f:
#     for i, line in enumerate(f):
#         if i >= 100:  # Test on first 100 samples
#             break
#         try:
#             item = json.loads(line)
#             if item.get('text') and len(item.get('markers', [])) > 0:
#                 samples.append(item)
#         except:
#             pass

# print(f"✅ Loaded {len(samples)} samples with markers")

# # Test different thresholds
# thresholds = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7]

# print(f"\n{'='*70}")
# print("TESTING THRESHOLDS")
# print("="*70)

# results = []

# for threshold in tqdm(thresholds, desc="Testing thresholds"):
#     total_true = 0
#     total_pred = 0
#     marker_counts = Counter()

#     for sample in samples:
#         true_markers = sample['markers']
#         pred_markers = predict_with_threshold(sample['text'], threshold=threshold)

#         total_true += len(true_markers)
#         total_pred += len(pred_markers)

#         for marker in pred_markers:
#             marker_counts[marker['type']] += 1

#     avg_pred = total_pred / len(samples)
#     avg_true = total_true / len(samples)

#     results.append({
#         'threshold': threshold,
#         'avg_pred': avg_pred,
#         'avg_true': avg_true,
#         'total_pred': total_pred,
#         'total_true': total_true,
#         'marker_counts': marker_counts
#     })

# # Display results
# print(f"\n{'='*70}")
# print("RESULTS")
# print("="*70)
# print(f"True average: {results[0]['avg_true']:.2f} markers/doc")
# print()

# print(f"{'Threshold':<12} {'Avg Pred':<10} {'Total':<8} {'Diff':<8} {'Recommendation'}")
# print("-"*70)

# best_threshold = None
# best_diff = float('inf')

# for r in results:
#     diff = abs(r['avg_pred'] - r['avg_true'])
#     recommendation = ""

#     if diff < best_diff:
#         best_diff = diff
#         best_threshold = r['threshold']
#         recommendation = "⭐ BEST"
#     elif abs(r['avg_pred'] - r['avg_true']) < r['avg_true'] * 0.2:
#         recommendation = "✅ Good"
#     elif r['avg_pred'] < r['avg_true'] * 0.5:
#         recommendation = "⚠️ Too low"
#     elif r['avg_pred'] > r['avg_true'] * 1.5:
#         recommendation = "⚠️ Too high"

#     print(f"{r['threshold']:<12.2f} {r['avg_pred']:<10.2f} {r['total_pred']:<8d} {diff:<8.2f} {recommendation}")

# # Detailed analysis of best threshold
# print(f"\n{'='*70}")
# print(f"RECOMMENDED THRESHOLD: {best_threshold}")
# print("="*70)

# best_result = [r for r in results if r['threshold'] == best_threshold][0]

# print(f"\nWith threshold {best_threshold}:")
# print(f"  Average predictions: {best_result['avg_pred']:.2f} markers/doc")
# print(f"  Average true: {best_result['avg_true']:.2f} markers/doc")
# print(f"  Difference: {abs(best_result['avg_pred'] - best_result['avg_true']):.2f}")

# print(f"\nMarker type distribution:")
# total_markers = sum(best_result['marker_counts'].values())
# for mtype in MARKER_TYPES:
#     count = best_result['marker_counts'].get(mtype, 0)
#     pct = count / total_markers * 100 if total_markers > 0 else 0
#     print(f"  {mtype:10s}: {count:4d} ({pct:5.2f}%)")

# # Show sample prediction with best threshold
# print(f"\n{'='*70}")
# print("SAMPLE PREDICTION WITH BEST THRESHOLD")
# print("="*70)

# sample = samples[0]
# pred_markers = predict_with_threshold(sample['text'], threshold=best_threshold)

# print(f"\nText: {sample['text'][:150]}...")
# print(f"\nTrue markers ({len(sample['markers'])}):")
# for m in sample['markers']:
#     print(f"  [{m['type']:8s}] \"{m.get('text', sample['text'][m['startIndex']:m['endIndex']])}\"")

# print(f"\nPredicted markers ({len(pred_markers)}):")
# for m in pred_markers:
#     print(f"  [{m['type']:8s}] \"{sample['text'][m['startIndex']:m['endIndex']]}\"")

# # Final recommendation
# print(f"\n{'='*70}")
# print("FINAL RECOMMENDATION")
# print("="*70)

# print(f"\n✨ Use threshold: {best_threshold}")
# print(f"\n📝 Update your submission script:")
# print(f"   Change: confidence_threshold={best_threshold}")
# print(f"\n📊 Expected results on test set (~938 docs):")
# print(f"   Total markers: ~{int(best_result['avg_pred'] * 938)}")
# print(f"   Avg markers/doc: ~{best_result['avg_pred']:.2f}")

# if best_result['avg_pred'] < 2:
#     print(f"\n⚠️  Still quite low! Consider threshold {best_threshold - 0.05:.2f} for more markers")
# elif best_result['avg_pred'] > 6:
#     print(f"\n⚠️  Might be too many! Consider threshold {best_threshold + 0.05:.2f} for fewer false positives")
# else:
#     print(f"\n✅ This looks good!")

# print("="*70)

In [ ]:
# """
# ENSEMBLE TRAINING SCRIPT
# Train 3 diverse models for better marker extraction
# Target: 0.30-0.40 F1 (up from 0.18)
# """

# import json
# import torch
# import numpy as np
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForTokenClassification,
#     Trainer,
#     TrainingArguments,
#     DataCollatorForTokenClassification,
#     EarlyStoppingCallback
# )
# from seqeval.metrics import f1_score as seqeval_f1
# from seqeval.scheme import IOB2
# from collections import Counter
# import warnings
# warnings.filterwarnings('ignore')

# # ===========================
# # CONFIGURATION
# # ===========================

# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# labels_list = ["O"]
# for mt in MARKER_TYPES:
#     labels_list.extend([f"B-{mt}", f"I-{mt}"])

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# # ===========================
# # ENSEMBLE CONFIGURATIONS
# # ===========================

# ENSEMBLE_CONFIGS = [
#     {
#         "name": "model_1_deberta_512",
#         "model_name": "microsoft/deberta-v3-base",
#         "max_length": 512,
#         "batch_size": 4,
#         "learning_rate": 1e-5,
#         "epochs": 12,
#         "description": "DeBERTa-v3 with 512 tokens (balanced)"
#     },
#     {
#         "name": "model_2_deberta_768",
#         "model_name": "microsoft/deberta-v3-base",
#         "max_length": 768,
#         "batch_size": 2,  # Smaller batch for longer sequences
#         "learning_rate": 8e-6,
#         "epochs": 10,
#         "description": "DeBERTa-v3 with 768 tokens (captures long-range dependencies)"
#     },
#     {
#         "name": "model_3_roberta_large",
#         "model_name": "roberta-large",
#         "max_length": 512,
#         "batch_size": 2,  # Smaller batch for large model
#         "learning_rate": 5e-6,
#         "epochs": 10,
#         "description": "RoBERTa-Large (different architecture)"
#     }
# ]

# # ===========================
# # DATA LOADING FUNCTIONS
# # ===========================

# def align_labels_advanced(text, markers, tokenizer, max_length):
#     """Advanced label alignment"""
#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=max_length,
#         padding='max_length',
#         return_offsets_mapping=True,
#         add_special_tokens=True
#     )

#     offset_mapping = encoding['offset_mapping']
#     labels = []

#     for token_start, token_end in offset_mapping:
#         if token_start == 0 and token_end == 0:
#             labels.append(-100)
#         else:
#             labels.append(label2id["O"])

#     sorted_markers = sorted(markers, key=lambda x: (x['startIndex'], x['endIndex']))

#     for marker in sorted_markers:
#         start_char = marker['startIndex']
#         end_char = marker['endIndex']
#         marker_type = marker['type']

#         if marker_type not in MARKER_TYPES:
#             continue

#         first_token_idx = None
#         last_token_idx = None

#         for idx, (token_start, token_end) in enumerate(offset_mapping):
#             if token_start == 0 and token_end == 0:
#                 continue

#             if token_start < end_char and token_end > start_char:
#                 overlap_start = max(token_start, start_char)
#                 overlap_end = min(token_end, end_char)
#                 overlap_ratio = (overlap_end - overlap_start) / (token_end - token_start)

#                 if overlap_ratio >= 0.4:
#                     if first_token_idx is None:
#                         first_token_idx = idx
#                     last_token_idx = idx

#         if first_token_idx is not None:
#             labels[first_token_idx] = label2id[f"B-{marker_type}"]

#             if last_token_idx is not None and last_token_idx > first_token_idx:
#                 for idx in range(first_token_idx + 1, last_token_idx + 1):
#                     if labels[idx] != -100:
#                         labels[idx] = label2id[f"I-{marker_type}"]

#     return labels


# def create_dataset(jsonl_path, tokenizer, max_length, min_length=30):
#     """Create dataset for training"""
#     all_input_ids = []
#     all_attention_masks = []
#     all_labels = []

#     stats = {'total': 0, 'kept': 0, 'total_markers': 0}

#     with open(jsonl_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             stats['total'] += 1

#             try:
#                 item = json.loads(line)
#                 text = item.get('text', '').strip()
#                 markers = item.get('markers', [])

#                 if len(text) < min_length:
#                     continue

#                 # For training, keep examples with markers
#                 if 'train' in jsonl_path and len(markers) == 0:
#                     continue

#                 stats['total_markers'] += len(markers)

#                 encoding = tokenizer(
#                     text,
#                     truncation=True,
#                     max_length=max_length,
#                     padding='max_length'
#                 )

#                 label_ids = align_labels_advanced(text, markers, tokenizer, max_length)

#                 all_input_ids.append(encoding['input_ids'])
#                 all_attention_masks.append(encoding['attention_mask'])
#                 all_labels.append(label_ids)
#                 stats['kept'] += 1

#             except:
#                 continue

#     print(f"   Kept: {stats['kept']}/{stats['total']} examples")
#     print(f"   Total markers: {stats['total_markers']}")
#     if stats['kept'] > 0:
#         print(f"   Avg markers/doc: {stats['total_markers'] / stats['kept']:.2f}")

#     return Dataset.from_dict({
#         'input_ids': all_input_ids,
#         'attention_mask': all_attention_masks,
#         'labels': all_labels
#     })


# # ===========================
# # METRICS
# # ===========================

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=2)

#     true_labels = []
#     pred_labels = []

#     for prediction, label in zip(predictions, labels):
#         true_seq = []
#         pred_seq = []

#         for pred_id, label_id in zip(prediction, label):
#             if label_id != -100:
#                 true_seq.append(id2label[label_id])
#                 pred_seq.append(id2label[pred_id])

#         if len(true_seq) > 0:
#             true_labels.append(true_seq)
#             pred_labels.append(pred_seq)

#     try:
#         overall_f1 = seqeval_f1(true_labels, pred_labels, mode='strict', scheme=IOB2)
#     except:
#         overall_f1 = 0.0

#     return {"overall_f1": overall_f1}


# # ===========================
# # TRAIN ENSEMBLE
# # ===========================

# def train_single_model(config_idx):
#     """Train a single model from ensemble"""

#     config = ENSEMBLE_CONFIGS[config_idx]

#     print("\n" + "="*70)
#     print(f"TRAINING MODEL {config_idx + 1}/3: {config['name']}")
#     print("="*70)
#     print(f"Description: {config['description']}")
#     print(f"Model: {config['model_name']}")
#     print(f"Max length: {config['max_length']}")
#     print(f"Batch size: {config['batch_size']}")
#     print("="*70)

#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(config['model_name'])

#     # Create datasets
#     print("\nLoading training data...")
#     train_dataset = create_dataset(
#         "train_rehydrated_v2.jsonl",
#         tokenizer,
#         config['max_length']
#     )

#     print("\nLoading validation data...")
#     val_dataset_temp = create_dataset(
#         "dev_rehydrated_v2.jsonl",
#         tokenizer,
#         config['max_length']
#     )

#     # Check if we need to split from training
#     val_has_markers = False
#     if len(val_dataset_temp) > 0:
#         for i in range(min(10, len(val_dataset_temp))):
#             labels = val_dataset_temp[i]['labels']
#             if any(l > 0 and l != -100 for l in labels):
#                 val_has_markers = True
#                 break

#     if not val_has_markers:
#         print("⚠️  Dev set has no markers, splitting from training...")
#         split = train_dataset.train_test_split(test_size=0.15, seed=42)
#         train_dataset = split['train']
#         val_dataset = split['test']
#     else:
#         val_dataset = val_dataset_temp

#     print(f"\nFinal split: {len(train_dataset)} train, {len(val_dataset)} val")

#     train_dataset.set_format("torch")
#     val_dataset.set_format("torch")

#     # Initialize model
#     print(f"\nInitializing {config['model_name']}...")
#     model = AutoModelForTokenClassification.from_pretrained(
#         config['model_name'],
#         num_labels=len(labels_list),
#         id2label=id2label,
#         label2id=label2id,
#         ignore_mismatched_sizes=True
#     )

#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir=f"./ensemble/{config['name']}",
#         eval_strategy="epoch",
#         save_strategy="epoch",
#         learning_rate=config['learning_rate'],
#         per_device_train_batch_size=config['batch_size'],
#         per_device_eval_batch_size=config['batch_size'] * 2,
#         num_train_epochs=config['epochs'],
#         weight_decay=0.01,
#         warmup_ratio=0.1,
#         load_best_model_at_end=True,
#         metric_for_best_model="overall_f1",
#         greater_is_better=True,
#         logging_steps=50,
#         report_to="none",
#         save_total_limit=2,
#         fp16=torch.cuda.is_available(),
#         gradient_accumulation_steps=4,
#         lr_scheduler_type="cosine",
#         seed=42 + config_idx  # Different seed for diversity
#     )

#     # Data collator
#     data_collator = DataCollatorForTokenClassification(
#         tokenizer=tokenizer,
#         padding=True,
#         label_pad_token_id=-100
#     )

#     # Trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=val_dataset,
#         data_collator=data_collator,
#         tokenizer=tokenizer,
#         compute_metrics=compute_metrics,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
#     )

#     # Train
#     print(f"\n🚀 Starting training...")
#     trainer.train()

#     # Save
#     save_path = f"./ensemble/{config['name']}/checkpoint-best"
#     trainer.save_model(save_path)
#     tokenizer.save_pretrained(save_path)

#     # Evaluate
#     eval_results = trainer.evaluate()

#     print(f"\n✅ Model {config_idx + 1} complete!")
#     print(f"   Overall F1: {eval_results['eval_overall_f1']:.4f}")
#     print(f"   Saved to: {save_path}")

#     return eval_results['eval_overall_f1']


# # ===========================
# # MAIN
# # ===========================

# if __name__ == "__main__":
#     print("\n" + "="*70)
#     print("ENSEMBLE TRAINING PIPELINE")
#     print("="*70)
#     print(f"Training {len(ENSEMBLE_CONFIGS)} diverse models")
#     print(f"Expected improvement: 0.18 → 0.30-0.40 F1")
#     print("="*70)

#     # Train all models
#     f1_scores = []

#     for i in range(len(ENSEMBLE_CONFIGS)):
#         try:
#             f1 = train_single_model(i)
#             f1_scores.append(f1)
#         except Exception as e:
#             print(f"\n❌ Model {i+1} failed: {e}")
#             f1_scores.append(0.0)
#             continue

#     # Summary
#     print("\n" + "="*70)
#     print("ENSEMBLE TRAINING COMPLETE!")
#     print("="*70)

#     for i, (config, f1) in enumerate(zip(ENSEMBLE_CONFIGS, f1_scores)):
#         emoji = "🎉" if f1 > 0.20 else "✅" if f1 > 0.15 else "⚠️"
#         print(f"{emoji} Model {i+1} ({config['name']}): F1 = {f1:.4f}")

#     avg_f1 = np.mean([f for f in f1_scores if f > 0])
#     print(f"\n📊 Average F1: {avg_f1:.4f}")
#     print(f"📊 Expected ensemble F1: {avg_f1 * 1.15:.4f} (+15% from averaging)")

#     print("\n" + "="*70)
#     print("NEXT STEPS:")
#     print("="*70)
#     print("1. Run the ensemble inference script")
#     print("2. This will combine predictions from all 3 models")
#     print("3. Expected final score: 0.30-0.40 F1")
#     print("="*70)

In [ ]:
# """
# SPACE-EFFICIENT ENSEMBLE INFERENCE
# Single-pass processing to avoid disk space issues
# """
# import json
# import os
# import zipfile
# import torch
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from tqdm import tqdm
# from collections import Counter
# import gc

# # ===========================
# # CONFIGURATION
# # ===========================
# TEST_FILE = "test_rehydrated_v2.jsonl"
# OUTPUT_ZIP = "/kaggle/working/submission.zip"

# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# labels_list = ["O"]
# for mt in MARKER_TYPES:
#     labels_list.extend([f"B-{mt}", f"I-{mt}"])

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# # Use ONLY the best model (not ensemble) to save space
# MODEL_CONFIG = {
#     "path": "./marker_results_advanced/checkpoint-best",
#     "model_name": "microsoft/deberta-v3-base",
#     "max_length": 512
# }

# print("="*70)
# print("SPACE-EFFICIENT MARKER EXTRACTION")
# print("="*70)

# # ===========================
# # LOAD MODEL
# # ===========================
# def load_model():
#     """Load single best model"""
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"\nDevice: {device}")
#     print(f"Loading: {MODEL_CONFIG['model_name']}")

#     tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG['model_name'])

#     model = AutoModelForTokenClassification.from_pretrained(
#         MODEL_CONFIG['path'],
#         num_labels=len(labels_list),
#         id2label=id2label,
#         label2id=label2id
#     )
#     model.to(device)
#     model.eval()

#     print("✅ Model loaded")

#     return tokenizer, model, device

# # ===========================
# # PREDICTION
# # ===========================
# def predict_markers(text, tokenizer, model, device, confidence_threshold=0.15):
#     """Predict markers for text"""

#     if not text or len(text.strip()) < 10:
#         return []

#     # Tokenize
#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=MODEL_CONFIG['max_length'],
#         padding='max_length',
#         return_offsets_mapping=True,
#         return_tensors='pt'
#     )

#     offset_mapping = encoding['offset_mapping'][0].cpu().numpy()

#     # Move to device
#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     # Predict
#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits[0].cpu().numpy()

#         predictions = np.argmax(logits, axis=1)
#         probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
#         confidences = np.max(probs, axis=1)

#     # Extract markers
#     markers = []
#     current_marker = None
#     current_start = None
#     current_end = None
#     current_conf = []

#     for idx, (pred_id, conf, (token_start, token_end)) in enumerate(zip(predictions, confidences, offset_mapping)):
#         if token_start == 0 and token_end == 0:
#             continue

#         label = id2label[pred_id]

#         if label.startswith('B-'):
#             # Save previous
#             if current_marker is not None and np.mean(current_conf) >= confidence_threshold:
#                 markers.append({
#                     'startIndex': int(current_start),
#                     'endIndex': int(current_end),
#                     'type': current_marker,
#                     'text': text[current_start:current_end]
#                 })

#             # Start new
#             current_marker = label[2:]
#             current_start = int(token_start)
#             current_end = int(token_end)
#             current_conf = [conf]

#         elif label.startswith('I-') and current_marker is not None:
#             if label[2:] == current_marker:
#                 current_end = int(token_end)
#                 current_conf.append(conf)

#         elif label == 'O':
#             if current_marker is not None and np.mean(current_conf) >= confidence_threshold:
#                 markers.append({
#                     'startIndex': int(current_start),
#                     'endIndex': int(current_end),
#                     'type': current_marker,
#                     'text': text[current_start:current_end]
#                 })
#             current_marker = None

#     # Don't forget last
#     if current_marker is not None and np.mean(current_conf) >= confidence_threshold:
#         markers.append({
#             'startIndex': int(current_start),
#             'endIndex': int(current_end),
#             'type': current_marker,
#             'text': text[current_start:current_end]
#         })

#     return markers

# # ===========================
# # MAIN
# # ===========================
# def main():
#     # Load model
#     print("\n" + "="*70)
#     print("LOADING MODEL")
#     print("="*70)

#     tokenizer, model, device = load_model()

#     # Load test data
#     print(f"\n📂 Loading: {TEST_FILE}")

#     if not os.path.exists(TEST_FILE):
#         print(f"❌ Not found: {TEST_FILE}")
#         return

#     test_data = []
#     with open(TEST_FILE, 'r') as f:
#         for line in f:
#             try:
#                 test_data.append(json.loads(line))
#             except:
#                 pass

#     print(f"✅ Loaded {len(test_data)} documents")

#     # Single-pass: Generate predictions and write to ZIP directly
#     print(f"\n🔄 Generating predictions and creating submission...")
#     print(f"   Writing to: {OUTPUT_ZIP}")

#     stats = {
#         'total_markers': 0,
#         'docs_with_markers': 0,
#         'marker_counts': Counter()
#     }

#     # Write directly to ZIP in streaming mode
#     with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
#         with zf.open("submission.jsonl", mode="w") as zf_file:

#             for item in tqdm(test_data, desc="Processing"):
#                 doc_id = item['_id']
#                 text = item.get('text', '')

#                 # Predict markers
#                 if text and len(text) >= 10:
#                     markers = predict_markers(text, tokenizer, model, device, confidence_threshold=0.15)
#                 else:
#                     markers = []

#                 # Update stats
#                 stats['total_markers'] += len(markers)
#                 if len(markers) > 0:
#                     stats['docs_with_markers'] += 1

#                 for marker in markers:
#                     stats['marker_counts'][marker['type']] += 1

#                 # Create output
#                 output = {
#                     "_id": doc_id,
#                     "markers": markers
#                 }

#                 # Write to ZIP
#                 line = json.dumps(output, ensure_ascii=False) + "\n"
#                 zf_file.write(line.encode("utf-8"))

#                 # Periodic cleanup to free memory
#                 if len(test_data) > 500 and (test_data.index(item) + 1) % 100 == 0:
#                     gc.collect()
#                     torch.cuda.empty_cache() if torch.cuda.is_available() else None

#     # Print statistics
#     print(f"\n{'='*70}")
#     print("SUBMISSION STATISTICS")
#     print("="*70)

#     total_docs = len(test_data)
#     total_markers = stats['total_markers']
#     docs_with_markers = stats['docs_with_markers']

#     print(f"Total documents: {total_docs}")
#     print(f"Documents with markers: {docs_with_markers} ({docs_with_markers/total_docs*100:.1f}%)")
#     print(f"Total markers: {total_markers}")
#     print(f"Avg markers/doc: {total_markers/total_docs:.2f}")

#     print(f"\nMarker Distribution:")
#     for mtype in MARKER_TYPES:
#         count = stats['marker_counts'].get(mtype, 0)
#         pct = count / total_markers * 100 if total_markers > 0 else 0
#         print(f"   {mtype:10s}: {count:5d} ({pct:5.2f}%)")

#     # File info
#     file_size = os.path.getsize(OUTPUT_ZIP) / 1024
#     print(f"\n✅ Submission created: {OUTPUT_ZIP}")
#     print(f"📦 File size: {file_size:.2f} KB")

#     # Expected performance
#     avg_markers = total_markers / total_docs
#     print(f"\n{'='*70}")
#     print("EXPECTED PERFORMANCE")
#     print("="*70)

#     if avg_markers >= 5:
#         print(f"Avg markers/doc: {avg_markers:.2f}")
#         print(f"Expected F1: 0.35-0.45 🎉")
#     elif avg_markers >= 3.5:
#         print(f"Avg markers/doc: {avg_markers:.2f}")
#         print(f"Expected F1: 0.25-0.35 ✅")
#     elif avg_markers >= 2.5:
#         print(f"Avg markers/doc: {avg_markers:.2f}")
#         print(f"Expected F1: 0.18-0.28 ⚠️")
#     else:
#         print(f"Avg markers/doc: {avg_markers:.2f}")
#         print(f"Expected F1: 0.10-0.20 ❌")
#         print(f"Tip: Lower confidence_threshold to get more markers")

#     print("="*70)

# if __name__ == "__main__":
#     main()

In [ ]:
# """
# SUBTASK 1: MARKER EXTRACTION - FULLY FIXED
# Handles empty dev set and improves training
# """
# import json
# import torch
# import numpy as np
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForTokenClassification,
#     Trainer,
#     TrainingArguments,
#     DataCollatorForTokenClassification
# )
# from seqeval.metrics import f1_score as seqeval_f1
# from seqeval.metrics import classification_report as seqeval_report
# from seqeval.scheme import IOB2
# from collections import Counter

# print("="*70)
# print("SUBTASK 1: MARKER EXTRACTION - FULLY FIXED")
# print("="*70)

# # ===========================
# # 1. BIO TAG SCHEME
# # ===========================
# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]
# labels_list = ["O"]
# for marker_type in MARKER_TYPES:
#     labels_list.append(f"B-{marker_type}")
#     labels_list.append(f"I-{marker_type}")

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# print(f"\nTotal labels: {len(labels_list)}")
# print(f"Labels: {labels_list}")

# # ===========================
# # 2. FIXED MARKER ALIGNMENT
# # ===========================
# def align_labels_with_tokens_improved(text, markers, tokenizer):
#     """
#     Improved alignment with better overlap detection
#     """
#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=512,  # Increased from 256
#         padding='max_length',
#         return_offsets_mapping=True,
#         add_special_tokens=True
#     )

#     offset_mapping = encoding['offset_mapping']
#     labels = []

#     for token_start, token_end in offset_mapping:
#         if token_start == 0 and token_end == 0:
#             labels.append(-100)  # Special tokens/padding
#         else:
#             labels.append(label2id["O"])

#     # Process markers (handle overlapping)
#     for marker in markers:
#         start_char = marker['startIndex']
#         end_char = marker['endIndex']
#         marker_type = marker['type']

#         if marker_type not in MARKER_TYPES:
#             continue

#         # Find all tokens that overlap with this marker
#         overlapping_tokens = []
#         for idx, (token_start, token_end) in enumerate(offset_mapping):
#             if token_start == 0 and token_end == 0:
#                 continue

#             # Token overlaps if there's any intersection
#             if token_start < end_char and token_end > start_char:
#                 overlap_start = max(token_start, start_char)
#                 overlap_end = min(token_end, end_char)
#                 overlap_ratio = (overlap_end - overlap_start) / (token_end - token_start)

#                 # Require at least 50% overlap
#                 if overlap_ratio >= 0.5:
#                     overlapping_tokens.append(idx)

#         # Assign B- and I- tags
#         if overlapping_tokens:
#             labels[overlapping_tokens[0]] = label2id[f"B-{marker_type}"]
#             for idx in overlapping_tokens[1:]:
#                 labels[idx] = label2id[f"I-{marker_type}"]

#     return labels

# def create_token_dataset(jsonl_path, tokenizer, require_markers=False):
#     """
#     Create dataset with option to filter for examples with markers
#     """
#     all_input_ids = []
#     all_attention_masks = []
#     all_labels = []

#     skipped = 0
#     skipped_no_markers = 0
#     total_markers = 0
#     label_counts = Counter()

#     print(f"\nProcessing {jsonl_path}...")

#     with open(jsonl_path, 'r', encoding='utf-8') as f:
#         for line_num, line in enumerate(f, 1):
#             try:
#                 item = json.loads(line)
#                 text = item.get('text', '').strip()
#                 markers = item.get('markers', [])

#                 # Skip empty texts
#                 if not text or len(text) < 10:
#                     skipped += 1
#                     continue

#                 # Skip if no markers and we require them
#                 if require_markers and len(markers) == 0:
#                     skipped_no_markers += 1
#                     continue

#                 total_markers += len(markers)

#                 # Tokenize
#                 encoding = tokenizer(
#                     text,
#                     truncation=True,
#                     max_length=512,
#                     padding='max_length'
#                 )

#                 # Align labels
#                 label_ids = align_labels_with_tokens_improved(text, markers, tokenizer)

#                 # Count labels
#                 for label_id in label_ids:
#                     if label_id != -100:
#                         label_counts[id2label[label_id]] += 1

#                 all_input_ids.append(encoding['input_ids'])
#                 all_attention_masks.append(encoding['attention_mask'])
#                 all_labels.append(label_ids)

#             except Exception as e:
#                 skipped += 1
#                 continue

#     print(f"✅ Loaded {len(all_input_ids)} examples")
#     print(f"   Skipped (general): {skipped}")
#     if require_markers:
#         print(f"   Skipped (no markers): {skipped_no_markers}")
#     print(f"📊 Total markers: {total_markers}")

#     if len(all_input_ids) > 0:
#         print(f"📊 Avg markers per example: {total_markers / len(all_input_ids):.2f}")

#     # Show label distribution
#     print(f"\n📊 Label distribution:")
#     for label, count in sorted(label_counts.items(), key=lambda x: -x[1])[:15]:
#         print(f"   {label:15s}: {count:7d}")

#     # Check marker labels
#     marker_label_count = sum(count for label, count in label_counts.items() if label != 'O')
#     if marker_label_count == 0:
#         print("\n⚠️  WARNING: No marker labels found!")
#     else:
#         print(f"\n✅ Found {marker_label_count} marker labels")
#         print(f"   Ratio: {marker_label_count / (marker_label_count + label_counts.get('O', 0)):.2%}")

#     return Dataset.from_dict({
#         'input_ids': all_input_ids,
#         'attention_mask': all_attention_masks,
#         'labels': all_labels
#     })

# # ===========================
# # 3. LOAD DATA - SMART SPLIT
# # ===========================
# print("\n" + "="*70)
# print("LOADING DATA")
# print("="*70)

# model_name = "roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

# # Load training data
# train_dataset = create_token_dataset("train_rehydrated.jsonl", tokenizer)

# # Check dev set
# print("\n" + "="*70)
# print("CHECKING VALIDATION SET")
# print("="*70)

# val_dataset_temp = create_token_dataset("dev_rehydrated.jsonl", tokenizer)

# # Calculate marker ratio in validation
# val_has_markers = False
# if len(val_dataset_temp) > 0:
#     # Check if validation has any marker labels
#     sample_labels = val_dataset_temp['labels'][:10]
#     for labels in sample_labels:
#         if any(l > 0 and l != -100 for l in labels):
#             val_has_markers = True
#             break

# if not val_has_markers or len(val_dataset_temp) < 20:
#     print("\n⚠️  CRITICAL: Validation set has no markers or is too small!")
#     print("   Solution: Splitting from training data (15% validation)")

#     # Split training data
#     split = train_dataset.train_test_split(test_size=0.15, seed=42)
#     train_dataset = split['train']
#     val_dataset = split['test']

#     print(f"\n✅ New split created:")
#     print(f"   Training: {len(train_dataset)} examples")
#     print(f"   Validation: {len(val_dataset)} examples")
# else:
#     val_dataset = val_dataset_temp
#     print(f"\n✅ Using provided validation set:")
#     print(f"   Training: {len(train_dataset)} examples")
#     print(f"   Validation: {len(val_dataset)} examples")

# # Set format
# train_dataset.set_format("torch")
# val_dataset.set_format("torch")

# # ===========================
# # 4. MODEL
# # ===========================
# print("\n" + "="*70)
# print("INITIALIZING MODEL")
# print("="*70)

# model = AutoModelForTokenClassification.from_pretrained(
#     model_name,
#     num_labels=len(labels_list),
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True
# )

# print(f"✅ Model initialized with {len(labels_list)} labels")

# # ===========================
# # 5. IMPROVED METRICS
# # ===========================
# def compute_metrics_robust(eval_pred):
#     """
#     Robust metric computation with detailed logging
#     """
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=2)

#     # Convert to label strings
#     true_labels = []
#     pred_labels = []

#     for prediction, label in zip(predictions, labels):
#         true_seq = []
#         pred_seq = []

#         for pred_id, label_id in zip(prediction, label):
#             if label_id != -100:
#                 true_seq.append(id2label[label_id])
#                 pred_seq.append(id2label[pred_id])

#         if len(true_seq) > 0:
#             true_labels.append(true_seq)
#             pred_labels.append(pred_seq)

#     # Calculate metrics
#     results = {}

#     # Overall F1
#     try:
#         overall_f1 = seqeval_f1(true_labels, pred_labels, mode='strict', scheme=IOB2)
#         results["overall_f1"] = overall_f1
#     except Exception as e:
#         print(f"Error computing overall F1: {e}")
#         results["overall_f1"] = 0.0

#     # Per-marker F1
#     for marker_type in MARKER_TYPES:
#         try:
#             # Filter for this marker type
#             filtered_true = []
#             filtered_pred = []

#             for true_seq, pred_seq in zip(true_labels, pred_labels):
#                 ft = []
#                 fp = []
#                 for t, p in zip(true_seq, pred_seq):
#                     if marker_type in t:
#                         ft.append(t)
#                         fp.append(p if marker_type in p else 'O')
#                     else:
#                         ft.append('O')
#                         fp.append('O')

#                 filtered_true.append(ft)
#                 filtered_pred.append(fp)

#             if any(any(x != 'O' for x in seq) for seq in filtered_true):
#                 marker_f1 = seqeval_f1(filtered_true, filtered_pred, mode='strict', scheme=IOB2)
#             else:
#                 marker_f1 = 0.0

#             results[f"f1_{marker_type}"] = marker_f1
#         except Exception as e:
#             results[f"f1_{marker_type}"] = 0.0

#     return results

# # ===========================
# # 6. DATA COLLATOR
# # ===========================
# data_collator = DataCollatorForTokenClassification(
#     tokenizer=tokenizer,
#     padding=True,
#     label_pad_token_id=-100
# )

# # ===========================
# # 7. IMPROVED TRAINING ARGS
# # ===========================
# training_args = TrainingArguments(
#     output_dir="./marker_results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,  # Balanced learning rate
#     per_device_train_batch_size=8,  # Smaller batch for better gradients
#     per_device_eval_batch_size=16,
#     num_train_epochs=10,  # More epochs
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     load_best_model_at_end=True,
#     metric_for_best_model="overall_f1",
#     greater_is_better=True,
#     logging_steps=50,
#     logging_first_step=True,
#     report_to="none",
#     save_total_limit=3,
#     fp16=torch.cuda.is_available(),
#     gradient_accumulation_steps=2,  # Effective batch = 16
#     dataloader_num_workers=2
# )

# # ===========================
# # 8. TRAINER
# # ===========================
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics_robust
# )

# # ===========================
# # 9. TRAIN
# # ===========================
# print("\n" + "="*70)
# print("STARTING TRAINING")
# print("="*70)
# print("📊 Expected metrics:")
# print("   Epoch 1-2: F1 0.00-0.05 (model learning patterns)")
# print("   Epoch 3-5: F1 0.05-0.15 (markers being detected)")
# print("   Epoch 6+:  F1 0.15-0.30 (good performance)")
# print("="*70 + "\n")

# trainer.train()

# # ===========================
# # 10. SAVE
# # ===========================
# print("\n" + "="*70)
# print("SAVING MODEL")
# print("="*70)

# trainer.save_model("./marker_results/checkpoint-best")
# tokenizer.save_pretrained("./marker_results/checkpoint-best")
# print("✅ Model saved to ./marker_results/checkpoint-best")

# # ===========================
# # 11. DETAILED EVALUATION
# # ===========================
# print("\n" + "="*70)
# print("FINAL EVALUATION")
# print("="*70)

# eval_results = trainer.evaluate()

# print(f"\n📊 OVERALL RESULTS:")
# print(f"   Overall F1: {eval_results['eval_overall_f1']:.4f}")

# print(f"\n📊 PER-MARKER F1 SCORES:")
# for marker_type in MARKER_TYPES:
#     key = f"eval_f1_{marker_type}"
#     if key in eval_results:
#         score = eval_results[key]
#         if score > 0.15:
#             emoji = "✅"
#         elif score > 0.08:
#             emoji = "⚠️"
#         else:
#             emoji = "❌"
#         print(f"   {emoji} {marker_type:10s}: {score:.4f}")

# # ===========================
# # 12. SAMPLE PREDICTIONS
# # ===========================
# print("\n" + "="*70)
# print("SAMPLE PREDICTIONS")
# print("="*70)

# predictions = trainer.predict(val_dataset)
# preds = np.argmax(predictions.predictions, axis=2)
# labels_array = predictions.label_ids

# # Convert to strings
# true_labels = []
# pred_labels = []

# for pred, label in zip(preds, labels_array):
#     true_seq = []
#     pred_seq = []
#     for p, l in zip(pred, label):
#         if l != -100:
#             true_seq.append(id2label[l])
#             pred_seq.append(id2label[p])
#     if len(true_seq) > 0:
#         true_labels.append(true_seq)
#         pred_labels.append(pred_seq)

# # Show examples with markers
# print("\nShowing examples with predicted markers:")
# shown = 0
# for i in range(min(10, len(true_labels))):
#     # Check if this example has any predicted markers
#     if any(label != 'O' for label in pred_labels[i]):
#         print(f"\nExample {shown + 1}:")
#         print(f"  True: {' '.join(true_labels[i][:30])}")
#         print(f"  Pred: {' '.join(pred_labels[i][:30])}")
#         shown += 1
#         if shown >= 3:
#             break

# # Classification report
# try:
#     print("\n" + "="*70)
#     print("CLASSIFICATION REPORT")
#     print("="*70)
#     report = seqeval_report(true_labels, pred_labels, mode='strict', scheme=IOB2, digits=4)
#     print(report)
# except Exception as e:
#     print(f"Could not generate report: {e}")

# # ===========================
# # 13. INTERPRETATION
# # ===========================
# print("\n" + "="*70)
# print("TRAINING COMPLETE!")
# print("="*70)

# final_f1 = eval_results['eval_overall_f1']

# if final_f1 > 0.25:
#     print("🎉 EXCELLENT! F1 > 0.25 - Top tier performance!")
# elif final_f1 > 0.18:
#     print("✅ GOOD! F1 > 0.18 - Competitive performance")
# elif final_f1 > 0.12:
#     print("⚠️  ACCEPTABLE - F1 > 0.12 - Room for improvement")
# elif final_f1 > 0.05:
#     print("⚠️  LOW - Model learning but needs tuning")
# else:
#     print("❌ VERY LOW - Check data alignment and quality")

# print("\n💡 TIPS TO IMPROVE:")
# print("   1. Increase max_length to 512 if texts are long")
# print("   2. Try DeBERTa-v3-base for better performance")
# print("   3. Increase epochs to 15 if still improving")
# print("   4. Use ensemble of multiple checkpoints")
# print("="*70)

In [ ]:
# # ===========================
# # DETAILED EVALUATION
# # ===========================
# print("\n" + "="*70)
# print("EVALUATING MODEL ON VALIDATION SET")
# print("="*70)

# # Get predictions
# predictions = trainer.predict(val_dataset)
# preds = np.argmax(predictions.predictions, axis=2)
# labels_array = predictions.label_ids

# # Convert to BIO sequences
# true_labels = []
# pred_labels = []

# for pred, label in zip(preds, labels_array):
#     true_seq = []
#     pred_seq = []
#     for p, l in zip(pred, label):
#         if l != -100:
#             true_seq.append(id2label[l])
#             pred_seq.append(id2label[p])
#     if len(true_seq) > 0:
#         true_labels.append(true_seq)
#         pred_labels.append(pred_seq)

# # Overall metrics
# overall_f1 = seqeval_f1(true_labels, pred_labels, mode='strict', scheme=IOB2)
# print(f"\n📊 Overall F1: {overall_f1:.4f}")

# # Per-marker metrics
# print(f"\n📊 Per-Marker F1 Scores:")
# for marker_type in MARKER_TYPES:
#     filtered_true = []
#     filtered_pred = []

#     for true_seq, pred_seq in zip(true_labels, pred_labels):
#         ft = [t if marker_type in t else 'O' for t in true_seq]
#         fp = [p if marker_type in p else 'O' for p in pred_seq]
#         filtered_true.append(ft)
#         filtered_pred.append(fp)

#     if any(any(x != 'O' for x in seq) for seq in filtered_true):
#         marker_f1 = seqeval_f1(filtered_true, filtered_pred, mode='strict', scheme=IOB2)
#         print(f"   {marker_type:10s}: {marker_f1:.4f}")

# # Show confusion examples
# print("\n📋 Sample Predictions:")
# for i in range(min(5, len(true_labels))):
#     if any(l != 'O' for l in true_labels[i][:20]):
#         print(f"\nExample {i+1}:")
#         print(f"  True: {' '.join(true_labels[i][:20])}")
#         print(f"  Pred: {' '.join(pred_labels[i][:20])}")

In [ ]:
# """
# SUBTASK 1: MARKER EXTRACTION - SUBMISSION GENERATOR
# Creates predictions for test set and packages them for submission
# """
# import json
# import os
# import zipfile
# import torch
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from tqdm import tqdm

# # ===========================
# # CONFIGURATION
# # ===========================
# # Paths
# INPUT_FILE = "/kaggle/input/test-reducted/test_rehydrated.jsonl"  # Update this
# MODEL_PATH = "./marker_results/checkpoint-best"
# OUTPUT_DIR = "/kaggle/working"
# TEMP_SUBMISSION_FILE = os.path.join(OUTPUT_DIR, "submission.jsonl")
# OUTPUT_ZIP = os.path.join(OUTPUT_DIR, "submission.zip")

# # Marker types
# MARKER_TYPES = ["Actor", "Action", "Effect", "Evidence", "Victim"]

# # Labels
# labels_list = ["O"]
# for marker_type in MARKER_TYPES:
#     labels_list.append(f"B-{marker_type}")
#     labels_list.append(f"I-{marker_type}")

# label2id = {label: i for i, label in enumerate(labels_list)}
# id2label = {i: label for label, i in label2id.items()}

# print("="*70)
# print("MARKER EXTRACTION - SUBMISSION GENERATOR")
# print("="*70)

# # ===========================
# # LOAD MODEL
# # ===========================
# def load_model():
#     """Load trained marker extraction model"""
#     print("\n📦 Loading model and tokenizer...")

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"   Device: {device}")

#     tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)

#     model = AutoModelForTokenClassification.from_pretrained(
#         MODEL_PATH,
#         num_labels=len(labels_list),
#         id2label=id2label,
#         label2id=label2id
#     )
#     model.to(device)
#     model.eval()

#     print("✅ Model loaded successfully!")
#     return tokenizer, model, device

# # ===========================
# # PREDICTION FUNCTION
# # ===========================
# def predict_markers(text, tokenizer, model, device):
#     """
#     Predict markers for a single text

#     Returns:
#         List of marker dictionaries with startIndex, endIndex, type, text
#     """
#     # Tokenize with offset mapping
#     encoding = tokenizer(
#         text,
#         truncation=True,
#         max_length=512,
#         padding='max_length',
#         return_offsets_mapping=True,
#         return_tensors='pt'
#     )

#     offset_mapping = encoding['offset_mapping'][0].numpy()

#     # Move to device
#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     # Predict
#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()

#     # Convert predictions to BIO tags
#     predicted_labels = []
#     for pred_id, (token_start, token_end) in zip(predictions, offset_mapping):
#         if token_start == 0 and token_end == 0:  # Special token
#             continue
#         predicted_labels.append(id2label[pred_id])

#     # Extract marker spans from BIO tags
#     markers = []
#     current_marker = None
#     current_start = None
#     current_end = None

#     for idx, label in enumerate(predicted_labels):
#         token_start, token_end = offset_mapping[idx]

#         if token_start == 0 and token_end == 0:
#             continue

#         if label.startswith('B-'):
#             # Save previous marker if exists
#             if current_marker is not None:
#                 markers.append({
#                     'startIndex': int(current_start),  # Convert to Python int
#                     'endIndex': int(current_end),      # Convert to Python int
#                     'type': current_marker,
#                     'text': text[current_start:current_end]
#                 })

#             # Start new marker
#             marker_type = label[2:]  # Remove 'B-'
#             current_marker = marker_type
#             current_start = int(token_start)  # Convert to Python int
#             current_end = int(token_end)      # Convert to Python int

#         elif label.startswith('I-') and current_marker is not None:
#             # Continue current marker
#             marker_type = label[2:]
#             if marker_type == current_marker:
#                 current_end = int(token_end)  # Convert to Python int
#             else:
#                 # Type mismatch, save current and start new
#                 markers.append({
#                     'startIndex': int(current_start),
#                     'endIndex': int(current_end),
#                     'type': current_marker,
#                     'text': text[current_start:current_end]
#                 })
#                 current_marker = marker_type
#                 current_start = int(token_start)
#                 current_end = int(token_end)

#         elif label == 'O':
#             # End current marker
#             if current_marker is not None:
#                 markers.append({
#                     'startIndex': int(current_start),
#                     'endIndex': int(current_end),
#                     'type': current_marker,
#                     'text': text[current_start:current_end]
#                 })
#                 current_marker = None

#     # Don't forget last marker
#     if current_marker is not None:
#         markers.append({
#             'startIndex': int(current_start),
#             'endIndex': int(current_end),
#             'type': current_marker,
#             'text': text[current_start:current_end]
#         })

#     return markers

# # ===========================
# # PROCESS DOCUMENTS
# # ===========================
# def process_document(item, tokenizer, model, device):
#     """Process a single document"""
#     doc_id = item.get('_id')
#     text = item.get('text', '')

#     if not doc_id or not text:
#         return None

#     # Predict markers
#     markers = predict_markers(text, tokenizer, model, device)

#     # Return in submission format
#     return {
#         "_id": doc_id,
#         "markers": markers
#     }

# # ===========================
# # MAIN EXECUTION
# # ===========================
# def main():
#     print("\n" + "="*70)
#     print("STARTING PREDICTION PIPELINE")
#     print("="*70)

#     # Load model
#     tokenizer, model, device = load_model()

#     # Load input data
#     print(f"\n📂 Loading test data from: {INPUT_FILE}")

#     if not os.path.exists(INPUT_FILE):
#         print(f"❌ Error: File not found: {INPUT_FILE}")
#         return

#     input_data = []
#     with open(INPUT_FILE, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 input_data.append(json.loads(line))
#             except json.JSONDecodeError:
#                 continue

#     print(f"✅ Loaded {len(input_data)} documents")

#     # Process all documents
#     print(f"\n🔄 Generating predictions...")
#     predictions = []

#     for item in tqdm(input_data, desc="Processing"):
#         pred = process_document(item, tokenizer, model, device)
#         if pred:
#             predictions.append(pred)

#     print(f"\n✅ Generated predictions for {len(predictions)} documents")

#     # Calculate statistics
#     total_markers = sum(len(pred['markers']) for pred in predictions)
#     print(f"\n📊 PREDICTION STATISTICS:")
#     print(f"   Total markers predicted: {total_markers}")
#     print(f"   Avg markers per document: {total_markers / len(predictions):.2f}")

#     # Marker type distribution
#     marker_counts = {mt: 0 for mt in MARKER_TYPES}
#     for pred in predictions:
#         for marker in pred['markers']:
#             marker_type = marker['type']
#             if marker_type in marker_counts:
#                 marker_counts[marker_type] += 1

#     print(f"\n📊 Marker Type Distribution:")
#     for marker_type, count in sorted(marker_counts.items(), key=lambda x: -x[1]):
#         percentage = (count / total_markers * 100) if total_markers > 0 else 0
#         print(f"   {marker_type:10s}: {count:5d} ({percentage:5.2f}%)")

#     # Save to JSONL
#     print(f"\n💾 Saving predictions to {TEMP_SUBMISSION_FILE}...")
#     with open(TEMP_SUBMISSION_FILE, 'w', encoding='utf-8') as f:
#         for pred in predictions:
#             f.write(json.dumps(pred) + '\n')

#     # Create ZIP
#     print(f"📦 Creating ZIP archive: {OUTPUT_ZIP}...")
#     with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zf:
#         zf.write(TEMP_SUBMISSION_FILE, arcname="submission.jsonl")

#     # Clean up
#     os.remove(TEMP_SUBMISSION_FILE)

#     file_size = os.path.getsize(OUTPUT_ZIP) / 1024
#     print(f"\n✅ SUCCESS!")
#     print(f"   Submission file: {OUTPUT_ZIP}")
#     print(f"   File size: {file_size:.2f} KB")

#     # Show sample predictions
#     print(f"\n📋 SAMPLE PREDICTIONS (first 3 documents):")
#     for i, pred in enumerate(predictions[:3]):
#         print(f"\nDocument {i+1} (ID: {pred['_id']}):")
#         if pred['markers']:
#             for marker in pred['markers'][:5]:  # Show first 5 markers
#                 print(f"   [{marker['type']:8s}] \"{marker['text'][:50]}...\"")
#         else:
#             print("   (No markers predicted)")

#     print("\n" + "="*70)
#     print("SUBMISSION READY FOR UPLOAD!")
#     print("="*70)

# if __name__ == "__main__":
#     main()

In [ ]:
# import torch
# import numpy as np
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# import json
# import torch
# import numpy as np
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     Trainer,
#     TrainingArguments
# )
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.metrics import f1_score, accuracy_score, classification_report

# # ===========================
# # 1. BINARY LABEL MAPPING
# # ===========================
# # ✅ Remove "Can't tell" - it's NOT a prediction class!
# label2id = {"No": 0, "Yes": 1}
# id2label = {0: "No", 1: "Yes"}

# # ===========================
# # 2. LOAD DATA - BINARY ONLY
# # ===========================
# def make_marker_text(text, markers):
#     """Wrap marker spans in special tokens."""
#     offset = 0
#     for m in sorted(markers, key=lambda x: x['startIndex']):
#         start = m['startIndex'] + offset
#         end = m['endIndex'] + offset
#         token_start = f"[{m['type'].upper()}]"
#         token_end = f"[/{m['type'].upper()}]"
#         text = text[:start] + token_start + text[start:end] + token_end + text[end:]
#         offset += len(token_start) + len(token_end)
#     return text


# def load_jsonl_binary(path):
#     """Load JSONL and convert to BINARY classification."""
#     texts, labels, raw_markers = [], [], []
#     skipped_cant_tell = 0
#     skipped_other = 0

#     with open(path, encoding="utf-8") as f:
#         for line in f:
#             item = json.loads(line)
#             label = item.get("conspiracy")

#             # ✅ SKIP "Can't tell" - not a valid class for prediction!
#             if label == "Can't tell":
#                 skipped_cant_tell += 1
#                 continue

#             # ✅ Only keep "Yes" and "No"
#             if label not in label2id:
#                 skipped_other += 1
#                 continue

#             # Marker-aware text
#             text_marker = make_marker_text(item["text"], item.get("markers", []))
#             texts.append(text_marker)
#             labels.append(label2id[label])
#             raw_markers.append(item.get("markers", []))

#     print(f"Loaded {len(texts)} samples from {path}")
#     print(f"  - Skipped 'Can't tell': {skipped_cant_tell}")
#     print(f"  - Skipped other: {skipped_other}")

#     return Dataset.from_dict({
#         "text": texts,
#         "label": labels,
#         "markers": raw_markers
#     })


# # Load datasets
# print("="*70)
# print("LOADING BINARY CLASSIFICATION DATA")
# print("="*70)

# train_ds = load_jsonl_binary("train_rehydrated.jsonl")
# dev_ds_raw = load_jsonl_binary("dev_rehydrated.jsonl")

# # Fallback if dev empty
# if len(dev_ds_raw) == 0:
#     print("\n⚠️  Dev set empty, splitting train set (90/10)")
#     split = train_ds.train_test_split(test_size=0.1, seed=42)
#     train_ds = split["train"]
#     dev_ds = split["test"]
# else:
#     dev_ds = dev_ds_raw

# # ===========================
# # 3. TOKENIZER SETUP
# # ===========================
# model_name = "roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Add special tokens for markers
# special_tokens = [
#     "[EVIDENCE]", "[/EVIDENCE]",
#     "[ACTOR]", "[/ACTOR]",
#     "[ACTION]", "[/ACTION]",
#     "[VICTIM]", "[/VICTIM]",
#     "[EFFECT]", "[/EFFECT]"
# ]
# tokenizer.add_tokens(special_tokens)


# def tokenize_fn(examples):
#     return tokenizer(
#         examples["text"],
#         padding="max_length",
#         truncation=True,
#         max_length=256
#     )


# train_ds = train_ds.map(tokenize_fn, batched=True)
# dev_ds = dev_ds.map(tokenize_fn, batched=True)

# train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# dev_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# # ===========================
# # 4. CLASS WEIGHTS
# # ===========================
# labels = np.array(train_ds["label"])
# class_weights = compute_class_weight(
#     class_weight="balanced",
#     classes=np.unique(labels),
#     y=labels
# )

# # Apply stronger weighting for minority class
# class_weights = class_weights ** 1.3  # Moderate boost
# class_weights = torch.tensor(class_weights, dtype=torch.float)

# print("\n" + "="*70)
# print("CLASS DISTRIBUTION & WEIGHTS")
# print("="*70)
# print(f"Class weights: {class_weights}")

# unique, counts = np.unique(labels, return_counts=True)
# print("\nTraining set distribution:")
# for label_id, count in zip(unique, counts):
#     pct = count / len(labels) * 100
#     print(f"  {id2label[label_id]:5s}: {count:5d} ({pct:5.2f}%)")

# # ===========================
# # 5. MODEL
# # ===========================
# print("\n" + "="*70)
# print("INITIALIZING MODEL")
# print("="*70)

# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=2,  # ✅ BINARY
#     id2label=id2label,
#     label2id=label2id
# )
# model.resize_token_embeddings(len(tokenizer))

# # ===========================
# # 6. WEIGHTED TRAINER
# # ===========================
# class WeightedTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
#         loss = loss_fn(logits, labels)
#         return (loss, outputs) if return_outputs else loss

# # ===========================
# # 7. METRICS
# # ===========================
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)

#     acc = accuracy_score(labels, preds)
#     f1_macro = f1_score(labels, preds, average="macro")
#     f1_binary = f1_score(labels, preds, average="binary", pos_label=1)  # "Yes" is positive

#     # Count predictions per class
#     unique, counts = np.unique(preds, return_counts=True)
#     pred_dist = {id2label[u]: c for u, c in zip(unique, counts)}

#     print(f"\nEval Predictions: {pred_dist}")

#     return {
#         "accuracy": acc,
#         "macro_f1": f1_macro,
#         "binary_f1": f1_binary  # F1 for "Yes" class
#     }

# # ===========================
# # 8. TRAINING ARGS
# # ===========================
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=4,  # Moderate epochs
#     weight_decay=0.01,
#     warmup_ratio=0.1,  # Warmup for 10% of training
#     load_best_model_at_end=True,
#     metric_for_best_model="macro_f1",
#     greater_is_better=True,
#     logging_steps=50,
#     report_to="none",
#     label_smoothing_factor=0.05,  # Light smoothing
#     save_total_limit=2,
#     fp16=True  # Mixed precision for faster training
# )

# # ===========================
# # 9. TRAIN
# # ===========================
# print("\n" + "="*70)
# print("STARTING TRAINING")
# print("="*70)

# trainer = WeightedTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=dev_ds,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# trainer.train()

# # ===========================
# # 10. SAVE BEST MODEL
# # ===========================
# print("\n" + "="*70)
# print("SAVING BEST MODEL")
# print("="*70)

# trainer.save_model("./results/checkpoint-best")
# tokenizer.save_pretrained("./results/checkpoint-best")
# print("✓ Model saved to ./results/checkpoint-best")

# # ===========================
# # 11. FINAL EVALUATION
# # ===========================
# print("\n" + "="*70)
# print("FINAL EVALUATION")
# print("="*70)

# eval_results = trainer.evaluate()
# print(f"\nAccuracy:   {eval_results['eval_accuracy']:.4f}")
# print(f"Macro F1:   {eval_results['eval_macro_f1']:.4f}")
# print(f"Binary F1:  {eval_results['eval_binary_f1']:.4f}")

# # Detailed classification report
# predictions = trainer.predict(dev_ds)
# pred_labels = np.argmax(predictions.predictions, axis=1)
# true_labels = predictions.label_ids

# print("\n" + "="*70)
# print("CLASSIFICATION REPORT")
# print("="*70)
# print(classification_report(
#     true_labels,
#     pred_labels,
#     target_names=["No", "Yes"],
#     digits=4
# ))

# # Check prediction distribution
# unique_pred, counts_pred = np.unique(pred_labels, return_counts=True)
# print("\nPrediction Distribution:")
# for u, c in zip(unique_pred, counts_pred):
#     pct = c / len(pred_labels) * 100
#     print(f"  {id2label[u]:5s}: {c:5d} ({pct:5.2f}%)")

# # Warning if heavily biased
# yes_pct = (np.sum(pred_labels == 1) / len(pred_labels)) * 100
# if yes_pct < 10:
#     print(f"\n⚠️  WARNING: Only {yes_pct:.1f}% 'Yes' predictions!")
#     print("   Model is heavily biased toward 'No'")
#     print("   Consider:")
#     print("   - Increasing class weights more")
#     print("   - Using focal loss")
#     print("   - Data augmentation for 'Yes' class")
# elif yes_pct > 90:
#     print(f"\n⚠️  WARNING: {yes_pct:.1f}% 'Yes' predictions!")
#     print("   Model is heavily biased toward 'Yes'")
# else:
#     print(f"\n✅ Reasonable balance: {yes_pct:.1f}% 'Yes' predictions")

# print("\n" + "="*70)
# print("TRAINING COMPLETE!")
# print("="*70)

In [ ]:
# import json
# import os
# import zipfile
# import torch
# import numpy as np
# from typing import List, Dict, Any
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # --- Configuration ---
# INPUT_FILE = "/kaggle/input/test-reducted/test_rehydrated.jsonl"
# OUTPUT_DIR = "/kaggle/working"
# TEMP_SUBMISSION_FILE = os.path.join(OUTPUT_DIR, "submission.jsonl")
# OUTPUT_ZIP = os.path.join(OUTPUT_DIR, "submission.zip")
# MODEL_PATH = "./results/checkpoint-best"
# MODEL_NAME = "roberta-base"

# # ✅ BINARY LABELS ONLY (No "Can't tell" in predictions!)
# id2label = {0: "No", 1: "Yes"}
# label2id = {"No": 0, "Yes": 1}

# def make_marker_text(text: str, markers: List[Dict]) -> str:
#     """Wrap marker spans in special tokens for marker-awareness."""
#     if not markers:
#         return text

#     offset = 0
#     for m in sorted(markers, key=lambda x: x['startIndex']):
#         start = m['startIndex'] + offset
#         end = m['endIndex'] + offset
#         token_start = f"[{m['type'].upper()}]"
#         token_end = f"[/{m['type'].upper()}]"
#         text = text[:start] + token_start + text[start:end] + token_end + text[end:]
#         offset += len(token_start) + len(token_end)
#     return text


# def load_model():
#     """Load the trained conspiracy classification model."""
#     print("Loading model and tokenizer...")

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")

#     tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#     # Add special tokens
#     special_tokens = ["[EVIDENCE]", "[/EVIDENCE]", "[ACTOR]", "[/ACTOR]",
#                       "[ACTION]", "[/ACTION]", "[VICTIM]", "[/VICTIM]",
#                       "[EFFECT]", "[/EFFECT]"]
#     tokenizer.add_tokens(special_tokens)

#     # Load model with BINARY labels
#     model = AutoModelForSequenceClassification.from_pretrained(
#         MODEL_PATH,
#         num_labels=2,  # ✅ BINARY ONLY
#         id2label=id2label,
#         label2id=label2id,
#         ignore_mismatched_sizes=True  # Allow loading 3-class model as 2-class
#     )
#     model.to(device)
#     model.eval()

#     print("Model loaded successfully!")
#     return tokenizer, model, device


# def predict_conspiracy(text: str, tokenizer, model, device,
#                        markers: List[Dict] = None,
#                        threshold: float = 0.5) -> str:
#     """
#     Predict conspiracy label using the trained model.
#     Uses probability threshold for better calibration.

#     Returns: "Yes" or "No" (NEVER "Can't tell")
#     """
#     # Apply marker-aware formatting if markers provided
#     if markers:
#         text = make_marker_text(text, markers)

#     # Tokenize
#     inputs = tokenizer(
#         text,
#         padding="max_length",
#         truncation=True,
#         max_length=256,
#         return_tensors="pt"
#     )

#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     # Predict with probability scores
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
#         probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

#     # Use threshold for better calibration
#     # If prob(Yes) > threshold, predict "Yes", else "No"
#     yes_prob = probs[1] if len(probs) > 1 else probs[0]

#     if yes_prob > threshold:
#         return "Yes"
#     else:
#         return "No"


# def predict_markers(doc_id: str, text: str) -> List[Dict[str, Any]]:
#     """
#     Placeholder for marker prediction.
#     For Subtask 2 (detection only), return empty list.
#     """
#     return []


# def process_document(item: Dict[str, Any], tokenizer, model, device) -> Dict[str, Any]:
#     """Process a single document and generate prediction."""

#     doc_id = item.get('_id')
#     text = item.get('text', '')

#     if not doc_id or not text:
#         return None

#     markers = item.get('markers', [])

#     # Predict conspiracy label (BINARY: Yes or No)
#     conspiracy_pred = predict_conspiracy(
#         text,
#         tokenizer,
#         model,
#         device,
#         markers if markers else None,
#         threshold=0.4  # Adjust threshold if needed (lower = more "Yes" predictions)
#     )

#     # Predict markers (empty for detection-only submission)
#     marker_preds = predict_markers(doc_id, text)

#     return {
#         "_id": doc_id,
#         "conspiracy": conspiracy_pred,  # ✅ Always "Yes" or "No"
#         "markers": marker_preds
#     }


# def load_jsonl(file_path: str) -> List[Dict[str, Any]]:
#     """Load JSONL file."""
#     data = []

#     if not os.path.exists(file_path):
#         raise FileNotFoundError(f"Input file not found: {file_path}")

#     print(f"Reading data from {file_path}...")
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line_num, line in enumerate(f, 1):
#             try:
#                 data.append(json.loads(line.strip()))
#             except json.JSONDecodeError:
#                 print(f"Warning: Skipping invalid JSON at line {line_num}")

#     print(f"Loaded {len(data)} documents")
#     return data


# def save_and_zip(data: List[Dict], temp_file: str, output_zip: str):
#     """Save predictions to JSONL and create ZIP."""

#     print(f"Saving {len(data)} predictions...")
#     with open(temp_file, 'w', encoding='utf-8') as f:
#         for item in data:
#             f.write(json.dumps(item) + '\n')

#     print(f"Creating ZIP: {output_zip}...")
#     with zipfile.ZipFile(OUTPUT_ZIP, 'w', compression=zipfile.ZIP_STORED) as zf:

#         zf.write(temp_file, arcname="submission.jsonl")

#     os.remove(temp_file)

#     print(f"✓ Submission created: {output_zip}")
#     print(f"  File size: {os.path.getsize(output_zip) / 1024:.2f} KB")


# def main():
#     """Main execution."""

#     print("="*70)
#     print("SEMEVAL 2026 TASK 10 - CONSPIRACY DETECTION SUBMISSION")
#     print("="*70)

#     # Load model
#     tokenizer, model, device = load_model()

#     # Load input data
#     input_data = load_jsonl(INPUT_FILE)

#     # Generate predictions
#     print(f"\nGenerating predictions for {len(input_data)} documents...")
#     final_submission = []

#     for i, item in enumerate(input_data):
#         if (i + 1) % 100 == 0:
#             print(f"  Progress: {i + 1}/{len(input_data)}")

#         prediction = process_document(item, tokenizer, model, device)
#         if prediction:
#             final_submission.append(prediction)

#     print(f"\n✓ Generated {len(final_submission)} predictions")

#     # Validate predictions
#     print("\n" + "="*70)
#     print("VALIDATION CHECK")
#     print("="*70)

#     labels = [doc['conspiracy'] for doc in final_submission]

#     # Check for invalid labels
#     invalid_labels = [l for l in labels if l not in ["Yes", "No"]]
#     if invalid_labels:
#         print(f"❌ ERROR: Found {len(invalid_labels)} invalid labels!")
#         print(f"   Invalid labels: {set(invalid_labels)}")
#         return
#     else:
#         print("✅ All labels are valid (Yes/No only)")

#     # Show distribution
#     print("\nPrediction Distribution:")
#     for label in ["Yes", "No"]:
#         count = labels.count(label)
#         percentage = (count / len(labels) * 100) if labels else 0
#         print(f"  {label:5s}: {count:5d} ({percentage:5.2f}%)")

#     # Check if model is too biased
#     yes_count = labels.count("Yes")
#     yes_pct = (yes_count / len(labels) * 100) if labels else 0

#     if yes_pct < 5:
#         print(f"\n⚠️  WARNING: Only {yes_pct:.1f}% 'Yes' predictions!")
#         print("   Your model may be too biased. Consider:")
#         print("   - Lowering threshold (currently 0.4)")
#         print("   - Retraining with different class weights")
#         print("   - Using a different model architecture")
#     elif yes_pct > 95:
#         print(f"\n⚠️  WARNING: {yes_pct:.1f}% 'Yes' predictions!")
#         print("   Your model may be overpredicting. Consider:")
#         print("   - Raising threshold")
#         print("   - Checking training data balance")
#     else:
#         print(f"\n✅ Prediction distribution looks reasonable ({yes_pct:.1f}% Yes)")

#     # Save submission
#     print("\n" + "="*70)
#     save_and_zip(final_submission, TEMP_SUBMISSION_FILE, OUTPUT_ZIP)

#     print("\n" + "="*70)
#     print("SUBMISSION READY FOR CODABENCH!")
#     print(f"File: {OUTPUT_ZIP}")
#     print("="*70)


# if __name__ == "__main__":
#     main()

In [ ]:
# # Check what your model actually learned
# from sklearn.metrics import confusion_matrix
# import numpy as np

# # Get predictions on dev set
# predictions = trainer.predict(dev_ds)
# pred_labels = np.argmax(predictions.predictions, axis=1)
# true_labels = predictions.label_ids

# # Confusion matrix
# cm = confusion_matrix(true_labels, pred_labels)
# print("Confusion Matrix:")
# print("         No  Can't  Yes")
# for i, label in enumerate(["No", "Can't", "Yes"]):
#     print(f"{label:8s} {cm[i]}")

# # Check if model ever predicts "Yes"
# yes_predictions = np.sum(pred_labels == 2)
# print(f"\nTotal 'Yes' predictions: {yes_predictions}")

In [ ]:
# """
# COMPLETE INFERENCE PIPELINE FOR SEMEVAL 2026 TASK 10
# Step 1: Rehydrate test set
# Step 2: Generate predictions
# Step 3: Create submission
# """

# import json
# import os
# import zipfile
# import torch
# import numpy as np
# import requests
# import time
# from tqdm import tqdm
# from typing import List, Dict, Any
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from bs4 import BeautifulSoup
# from markdown import markdown
# import re

# # =============================
# # STEP 1: REHYDRATE TEST DATA
# # =============================

# def markdown_to_text(markdown_string):
#     """Converts a markdown string to plaintext"""
#     html = markdown(markdown_string)
#     html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
#     html = re.sub(r'<code>(.*?)</code>', ' ', html)
#     soup = BeautifulSoup(html, "html.parser")
#     text = ' '.join(soup.find_all(string=True))
#     return text


# def replace_urls(x, url_replacement_token='[URL]'):
#     return re.sub(r"http\S+", url_replacement_token, x)


# def replace_ss_prefix(x):
#     return re.sub(
#         r'^\W*(summary statement|submission statement|ss)[^a-zA-Z]*',
#         "", x, flags=re.I | re.U
#     ).strip()


# def preprocess(x):
#     return replace_ss_prefix(replace_urls(markdown_to_text(x)))


# def fetch_from_arctic_shift(comment_ids, max_retries=2, timeout=15):
#     """Fetch comments from Arctic Shift API"""
#     url = "https://arctic-shift.photon-reddit.com/api/comments/ids"
#     params = {
#         "ids": ",".join(comment_ids),
#         "fields": "body,subreddit,id"
#     }

#     for attempt in range(max_retries):
#         try:
#             with requests.Session() as session:
#                 session.headers.update({'Connection': 'close'})
#                 response = session.get(url, params=params, timeout=timeout)
#                 response.raise_for_status()
#                 data = response.json()

#                 if "data" in data and isinstance(data["data"], list):
#                     return {
#                         item['id']: item
#                         for item in data["data"]
#                         if item.get('body', '[deleted]').strip() not in ['[deleted]', '[removed]', '']
#                     }
#         except Exception:
#             if attempt < max_retries - 1:
#                 time.sleep(1)

#     return {}


# def rehydrate_test_data(input_file, output_file):
#     """
#     Rehydrate the test_redacted.jsonl file
#     CRITICAL: This must be done before inference!
#     """
#     print("\n" + "="*70)
#     print("STEP 1: REHYDRATING TEST DATA")
#     print("="*70)

#     if not os.path.exists(input_file):
#         print(f"❌ ERROR: Test file not found: {input_file}")
#         print(f"   Please ensure you have the test_redacted.jsonl file!")
#         return False

#     # Check if already rehydrated
#     if os.path.exists(output_file):
#         print(f"⚠️  {output_file} already exists!")
#         user_input = input("Rehydrate again? (y/n): ").lower()
#         if user_input != 'y':
#             print("Using existing rehydrated file...")
#             return True

#     # Load test IDs
#     items_to_process = []
#     original_data_map = {}

#     with open(input_file, encoding='utf-8') as f:
#         for line in f:
#             try:
#                 item = json.loads(line)
#                 if '_id' in item and item['_id'].startswith('t1_'):
#                     items_to_process.append(item['_id'])
#                     original_data_map[item['_id']] = item
#             except:
#                 pass

#     print(f"📊 Found {len(items_to_process)} test comments to rehydrate")

#     if len(items_to_process) == 0:
#         print("❌ No valid IDs found in test file!")
#         return False

#     # Rehydrate in batches
#     batch_size = 20
#     rehydrated_data = []
#     deleted_count = 0

#     for i in tqdm(range(0, len(items_to_process), batch_size), desc="Rehydrating"):
#         batch_full_ids = items_to_process[i:i + batch_size]
#         batch_clean_ids = [x[3:] for x in batch_full_ids]

#         rehydrated_map = fetch_from_arctic_shift(batch_clean_ids)

#         for fid in batch_full_ids:
#             cid = fid[3:]
#             if cid in rehydrated_map:
#                 r = rehydrated_map[cid]
#                 o = original_data_map[fid]
#                 rehydrated_data.append({
#                     "_id": fid,
#                     "text": preprocess(r["body"]),
#                     "subreddit": r["subreddit"],
#                     "conspiracy": o.get("conspiracy"),
#                     "markers": o.get("markers", []),
#                     "annotator": o.get("annotator")
#                 })
#             else:
#                 deleted_count += 1

#         time.sleep(0.8)

#     # Save rehydrated data
#     with open(output_file, "w", encoding="utf-8") as f:
#         for x in rehydrated_data:
#             f.write(json.dumps(x, ensure_ascii=False) + "\n")

#     success_rate = len(rehydrated_data) / len(items_to_process) * 100
#     print(f"\n✅ Rehydrated {len(rehydrated_data)}/{len(items_to_process)} ({success_rate:.1f}%)")
#     print(f"🗑️  Deleted/removed: {deleted_count}")

#     if success_rate < 50:
#         print("\n⚠️  WARNING: Low success rate!")
#         print("   Consider using a VPN or contacting organizers")

#     return True


# # =============================
# # STEP 2: LOAD MODEL & PREDICT
# # =============================

# def make_marker_text(text: str, markers: List[Dict]) -> str:
#     """Wrap marker spans in special tokens"""
#     if not markers:
#         return text

#     offset = 0
#     for m in sorted(markers, key=lambda x: x['startIndex']):
#         start = m['startIndex'] + offset
#         end = m['endIndex'] + offset
#         token_start = f"[{m['type'].upper()}]"
#         token_end = f"[/{m['type'].upper()}]"
#         text = text[:start] + token_start + text[start:end] + token_end + text[end:]
#         offset += len(token_start) + len(token_end)

#     return text


# def load_model_for_inference(model_path, model_name="roberta-base"):
#     """Load trained model for inference"""
#     print("\n" + "="*70)
#     print("STEP 2: LOADING MODEL")
#     print("="*70)

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Device: {device}")

#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     # Add special tokens
#     special_tokens = [
#         "[EVIDENCE]", "[/EVIDENCE]",
#         "[ACTOR]", "[/ACTOR]",
#         "[ACTION]", "[/ACTION]",
#         "[VICTIM]", "[/VICTIM]",
#         "[EFFECT]", "[/EFFECT]"
#     ]
#     tokenizer.add_tokens(special_tokens)

#     # Load model
#     id2label = {0: "No", 1: "Yes"}
#     label2id = {"No": 0, "Yes": 1}

#     model = AutoModelForSequenceClassification.from_pretrained(
#         model_path,
#         num_labels=2,
#         id2label=id2label,
#         label2id=label2id,
#         ignore_mismatched_sizes=True
#     )
#     model.to(device)
#     model.eval()

#     print("✅ Model loaded successfully!")
#     return tokenizer, model, device


# def predict_single(text: str, markers: List[Dict], tokenizer, model, device):
#     """Predict conspiracy label for a single text"""

#     # Apply marker formatting
#     if markers:
#         text = make_marker_text(text, markers)

#     # Tokenize
#     inputs = tokenizer(
#         text,
#         padding="max_length",
#         truncation=True,
#         max_length=256,
#         return_tensors="pt"
#     )

#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     # Predict
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
#         probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

#     # Get prediction
#     pred_id = np.argmax(probs)
#     pred_label = "Yes" if pred_id == 1 else "No"

#     return pred_label


# # =============================
# # STEP 3: GENERATE SUBMISSION
# # =============================

# def generate_submission(test_file, output_zip, model_path):
#     """Generate submission file"""
#     print("\n" + "="*70)
#     print("STEP 3: GENERATING PREDICTIONS")
#     print("="*70)

#     # Load model
#     tokenizer, model, device = load_model_for_inference(model_path)

#     # Load test data
#     print(f"\nLoading test data from: {test_file}")
#     test_data = []

#     with open(test_file, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 test_data.append(json.loads(line))
#             except:
#                 pass

#     print(f"Loaded {len(test_data)} test examples")

#     # Generate predictions
#     predictions = []

#     print("\nGenerating predictions...")
#     for item in tqdm(test_data):
#         doc_id = item['_id']
#         text = item.get('text', '')
#         markers = item.get('markers', [])

#         if not text:
#             # If no text, predict "No" as fallback
#             pred = "No"
#         else:
#             pred = predict_single(text, markers, tokenizer, model, device)

#         predictions.append({
#             "_id": doc_id,
#             "conspiracy": pred,
#             "markers": []  # Empty for detection task
#         })

#     # Validate predictions
#     print("\n" + "="*70)
#     print("VALIDATION")
#     print("="*70)

#     labels = [p['conspiracy'] for p in predictions]
#     yes_count = labels.count("Yes")
#     no_count = labels.count("No")
#     yes_pct = yes_count / len(labels) * 100

#     print(f"Total predictions: {len(predictions)}")
#     print(f"  Yes: {yes_count} ({yes_pct:.2f}%)")
#     print(f"  No:  {no_count} ({100-yes_pct:.2f}%)")

#     # Check for invalid labels
#     invalid = [l for l in labels if l not in ["Yes", "No"]]
#     if invalid:
#         print(f"\n❌ ERROR: Found {len(invalid)} invalid labels!")
#         return False

#     print("\n✅ All predictions are valid")

#     # Save submission
#     print("\n" + "="*70)
#     print("CREATING SUBMISSION FILE")
#     print("="*70)

#     temp_file = "submission.jsonl"

#     with open(temp_file, 'w', encoding='utf-8') as f:
#         for pred in predictions:
#             f.write(json.dumps(pred) + '\n')

#     # Create ZIP
#     with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
#         zf.write(temp_file, arcname="submission.jsonl")

#     os.remove(temp_file)

#     print(f"✅ Submission created: {output_zip}")
#     print(f"   File size: {os.path.getsize(output_zip) / 1024:.2f} KB")

#     return True


# # =============================
# # MAIN EXECUTION
# # =============================

# def main():
#     """Complete inference pipeline"""

#     print("\n" + "="*70)
#     print("SEMEVAL 2026 TASK 10 - COMPLETE INFERENCE PIPELINE")
#     print("="*70)

#     # Configuration - UPDATE THESE PATHS!
#     TEST_REDACTED = "/kaggle/input/test-reducted/test_rehydrated.jsonl"
#     TEST_REHYDRATED = "test_rehydrated.jsonl"
#     MODEL_PATH = "./results/checkpoint-best"
#     OUTPUT_ZIP = "/kaggle/working/submission.zip"

#     # Step 1: Rehydrate test data
#     success = rehydrate_test_data(TEST_REDACTED, TEST_REHYDRATED)

#     if not success:
#         print("\n❌ Rehydration failed! Cannot proceed.")
#         print("\nOptions:")
#         print("1. Try again with VPN")
#         print("2. Contact organizers for pre-rehydrated test set")
#         print("3. Check if test file path is correct")
#         return

#     # Step 2 & 3: Generate predictions and create submission
#     success = generate_submission(TEST_REHYDRATED, OUTPUT_ZIP, MODEL_PATH)

#     if success:
#         print("\n" + "="*70)
#         print("✅ SUBMISSION READY!")
#         print("="*70)
#         print(f"\nFile: {OUTPUT_ZIP}")
#         print("\nNext steps:")
#         print("1. Download submission.zip")
#         print("2. Go to Codabench")
#         print("3. Upload to 'My Submissions'")
#         print("4. Wait for evaluation")
#         print("="*70)
#     else:
#         print("\n❌ Submission generation failed!")


# if __name__ == "__main__":
#     main()

In [ ]:
# """
# DIAGNOSTIC TOOL: Find out why your submission scored 0.00
# """

# import json
# import os
# import zipfile
# from collections import Counter

# def check_submission_file(submission_path):
#     """
#     Analyze a submission file to find issues
#     """
#     print("\n" + "="*70)
#     print("DIAGNOSTIC: CHECKING SUBMISSION FILE")
#     print("="*70)

#     # Check if file exists
#     if not os.path.exists(submission_path):
#         print(f"❌ ERROR: File not found: {submission_path}")
#         return False

#     print(f"✅ File exists: {submission_path}")
#     print(f"   Size: {os.path.getsize(submission_path) / 1024:.2f} KB")

#     # If it's a ZIP, extract it first
#     if submission_path.endswith('.zip'):
#         print("\n📦 Extracting ZIP file...")
#         try:
#             with zipfile.ZipFile(submission_path, 'r') as zf:
#                 file_list = zf.namelist()
#                 print(f"   Files in ZIP: {file_list}")

#                 if 'submission.jsonl' not in file_list:
#                     print("❌ ERROR: submission.jsonl not found in ZIP!")
#                     return False

#                 # Extract to temp location
#                 zf.extract('submission.jsonl', '/tmp/')
#                 jsonl_path = '/tmp/submission.jsonl'
#                 print("✅ Extracted submission.jsonl")
#         except Exception as e:
#             print(f"❌ ERROR extracting ZIP: {e}")
#             return False
#     else:
#         jsonl_path = submission_path

#     # Analyze JSONL content
#     print("\n📊 ANALYZING CONTENT...")

#     try:
#         data = []
#         with open(jsonl_path, 'r', encoding='utf-8') as f:
#             for line_num, line in enumerate(f, 1):
#                 try:
#                     item = json.loads(line.strip())
#                     data.append(item)
#                 except json.JSONDecodeError:
#                     print(f"⚠️  Invalid JSON at line {line_num}")

#         print(f"✅ Loaded {len(data)} predictions")

#         if len(data) == 0:
#             print("❌ ERROR: Submission file is empty!")
#             return False

#         # Check first few entries
#         print("\n📝 FIRST 3 ENTRIES:")
#         for i, item in enumerate(data[:3], 1):
#             print(f"\n{i}. {json.dumps(item, indent=2)}")

#         # Check required fields
#         print("\n🔍 CHECKING REQUIRED FIELDS...")

#         missing_id = 0
#         missing_conspiracy = 0
#         missing_markers = 0

#         for item in data:
#             if '_id' not in item:
#                 missing_id += 1
#             if 'conspiracy' not in item:
#                 missing_conspiracy += 1
#             if 'markers' not in item:
#                 missing_markers += 1

#         if missing_id > 0:
#             print(f"❌ {missing_id} entries missing '_id' field!")
#         else:
#             print("✅ All entries have '_id'")

#         if missing_conspiracy > 0:
#             print(f"❌ {missing_conspiracy} entries missing 'conspiracy' field!")
#         else:
#             print("✅ All entries have 'conspiracy'")

#         if missing_markers > 0:
#             print(f"❌ {missing_markers} entries missing 'markers' field!")
#         else:
#             print("✅ All entries have 'markers'")

#         # Check ID format
#         print("\n🆔 CHECKING ID FORMAT...")
#         id_formats = Counter([item['_id'][:3] if '_id' in item else 'MISSING' for item in data])
#         print(f"   ID prefixes: {dict(id_formats)}")

#         if 't1_' not in id_formats:
#             print("❌ WARNING: No 't1_' prefixes found!")
#             print("   Test data should have comment IDs starting with 't1_'")

#         # Check conspiracy labels
#         print("\n🏷️  CHECKING CONSPIRACY LABELS...")
#         labels = [item.get('conspiracy', 'MISSING') for item in data]
#         label_counts = Counter(labels)

#         print("   Label distribution:")
#         for label, count in label_counts.most_common():
#             pct = count / len(labels) * 100
#             print(f"     {label:12s}: {count:5d} ({pct:5.2f}%)")

#         # Check for invalid labels
#         invalid_labels = [l for l in labels if l not in ['Yes', 'No']]
#         if invalid_labels:
#             print(f"\n❌ ERROR: Found {len(invalid_labels)} invalid labels!")
#             print(f"   Invalid values: {set(invalid_labels)}")
#             print("   Only 'Yes' and 'No' are allowed!")
#         else:
#             print("\n✅ All labels are valid (Yes/No only)")

#         # Check marker format
#         print("\n📍 CHECKING MARKERS...")

#         total_markers = sum(len(item.get('markers', [])) for item in data)
#         items_with_markers = sum(1 for item in data if len(item.get('markers', [])) > 0)

#         print(f"   Total markers: {total_markers}")
#         print(f"   Items with markers: {items_with_markers}/{len(data)}")

#         if items_with_markers > 0:
#             print("   ℹ️  Note: Markers are optional for detection task")

#         # Final verdict
#         print("\n" + "="*70)
#         print("DIAGNOSTIC SUMMARY")
#         print("="*70)

#         issues = []

#         if len(data) == 0:
#             issues.append("Empty submission file")
#         if missing_id > 0:
#             issues.append(f"{missing_id} missing '_id' fields")
#         if missing_conspiracy > 0:
#             issues.append(f"{missing_conspiracy} missing 'conspiracy' fields")
#         if invalid_labels:
#             issues.append(f"{len(invalid_labels)} invalid labels")
#         if 't1_' not in id_formats:
#             issues.append("Wrong ID format (not starting with 't1_')")

#         if issues:
#             print("❌ ISSUES FOUND:")
#             for i, issue in enumerate(issues, 1):
#                 print(f"   {i}. {issue}")
#             print("\nLikely cause of 0.00 score!")
#             return False
#         else:
#             print("✅ No obvious issues found!")
#             print("\nIf still getting 0.00, possible reasons:")
#             print("1. Test set IDs don't match what Codabench expects")
#             print("2. You uploaded to wrong task (detection vs extraction)")
#             print("3. Codabench server issue - try resubmitting")
#             print("4. Check if you need to rehydrate test data first")
#             return True

#     except Exception as e:
#         print(f"❌ ERROR analyzing file: {e}")
#         return False


# def compare_with_expected(submission_path, expected_test_path=None):
#     """
#     Compare your submission IDs with expected test IDs
#     """
#     print("\n" + "="*70)
#     print("COMPARING WITH TEST DATA")
#     print("="*70)

#     if not expected_test_path or not os.path.exists(expected_test_path):
#         print("⚠️  Test file not provided or not found")
#         print("   Skipping ID comparison")
#         return

#     # Load submission IDs
#     submission_ids = set()

#     if submission_path.endswith('.zip'):
#         with zipfile.ZipFile(submission_path, 'r') as zf:
#             with zf.open('submission.jsonl') as f:
#                 for line in f:
#                     try:
#                         item = json.loads(line)
#                         submission_ids.add(item.get('_id'))
#                     except:
#                         pass
#     else:
#         with open(submission_path, 'r') as f:
#             for line in f:
#                 try:
#                     item = json.loads(line)
#                     submission_ids.add(item.get('_id'))
#                 except:
#                     pass

#     # Load expected IDs
#     expected_ids = set()
#     with open(expected_test_path, 'r') as f:
#         for line in f:
#             try:
#                 item = json.loads(line)
#                 expected_ids.add(item.get('_id'))
#             except:
#                 pass

#     # Compare
#     print(f"Submission IDs: {len(submission_ids)}")
#     print(f"Expected IDs:   {len(expected_ids)}")

#     missing = expected_ids - submission_ids
#     extra = submission_ids - expected_ids

#     if missing:
#         print(f"\n⚠️  {len(missing)} IDs in test set but NOT in submission!")
#         print(f"   First 5 missing: {list(missing)[:5]}")

#     if extra:
#         print(f"\n⚠️  {len(extra)} IDs in submission but NOT in test set!")
#         print(f"   First 5 extra: {list(extra)[:5]}")

#     if not missing and not extra:
#         print("\n✅ All IDs match perfectly!")


# if __name__ == "__main__":

#     # UPDATE THIS PATH to your submission file
#     SUBMISSION_PATH = "/kaggle/working/submission.zip"

#     # Optional: path to test_redacted.jsonl to compare IDs
#     TEST_PATH = "/kaggle/input/test-reducted/test_rehydrated.jsonl"

#     print("\n" + "="*70)
#     print("SUBMISSION DIAGNOSTIC TOOL")
#     print("="*70)

#     # Check submission
#     is_valid = check_submission_file(SUBMISSION_PATH)

#     # Compare with test data
#     if os.path.exists(TEST_PATH):
#         compare_with_expected(SUBMISSION_PATH, TEST_PATH)

#     print("\n" + "="*70)
#     if is_valid:
#         print("✅ Submission appears valid")
#         print("   If still getting 0.00, contact organizers!")
#     else:
#         print("❌ Issues found - fix them and regenerate submission")
#     print("="*70 + "\n")

In [ ]:
# """
# DATA DIAGNOSTIC TOOL
# Run this BEFORE training to catch issues early
# """

# import json
# from collections import Counter

# def diagnose_dataset(file_path):
#     """
#     Comprehensive dataset diagnostics
#     """
#     print("\n" + "="*70)
#     print(f"DIAGNOSING: {file_path}")
#     print("="*70)

#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             data = [json.loads(line) for line in f]
#     except FileNotFoundError:
#         print(f"❌ File not found: {file_path}")
#         return
#     except Exception as e:
#         print(f"❌ Error loading file: {e}")
#         return

#     print(f"\n✅ Loaded {len(data)} entries")

#     # Basic checks
#     has_id = sum(1 for item in data if '_id' in item)
#     has_text = sum(1 for item in data if 'text' in item and item['text'])
#     has_markers = sum(1 for item in data if 'markers' in item and len(item['markers']) > 0)
#     has_conspiracy = sum(1 for item in data if 'conspiracy' in item)

#     print(f"\n📊 FIELD COVERAGE:")
#     print(f"   Has '_id':       {has_id}/{len(data)} ({has_id/len(data)*100:.1f}%)")
#     print(f"   Has 'text':      {has_text}/{len(data)} ({has_text/len(data)*100:.1f}%)")
#     print(f"   Has markers:     {has_markers}/{len(data)} ({has_markers/len(data)*100:.1f}%)")
#     print(f"   Has conspiracy:  {has_conspiracy}/{len(data)} ({has_conspiracy/len(data)*100:.1f}%)")

#     # Text length stats
#     text_lengths = [len(item.get('text', '')) for item in data if item.get('text')]
#     if text_lengths:
#         print(f"\n📏 TEXT LENGTH:")
#         print(f"   Min:     {min(text_lengths)} chars")
#         print(f"   Max:     {max(text_lengths)} chars")
#         print(f"   Average: {sum(text_lengths)/len(text_lengths):.1f} chars")
#         print(f"   Empty:   {sum(1 for l in text_lengths if l == 0)}")

#     # Marker statistics
#     total_markers = sum(len(item.get('markers', [])) for item in data)
#     marker_types = Counter()
#     marker_lengths = []

#     for item in data:
#         for marker in item.get('markers', []):
#             marker_types[marker.get('type', 'Unknown')] += 1
#             start = marker.get('startIndex', 0)
#             end = marker.get('endIndex', 0)
#             marker_lengths.append(end - start)

#     print(f"\n🎯 MARKERS:")
#     print(f"   Total markers: {total_markers}")
#     if has_markers > 0:
#         print(f"   Avg per doc:   {total_markers / has_markers:.2f}")

#     if marker_types:
#         print(f"\n📊 MARKER TYPES:")
#         for mtype, count in marker_types.most_common():
#             pct = count / total_markers * 100 if total_markers > 0 else 0
#             print(f"   {mtype:12s}: {count:5d} ({pct:5.2f}%)")

#     if marker_lengths:
#         print(f"\n📏 MARKER LENGTHS:")
#         print(f"   Min:     {min(marker_lengths)} chars")
#         print(f"   Max:     {max(marker_lengths)} chars")
#         print(f"   Average: {sum(marker_lengths)/len(marker_lengths):.1f} chars")

#     # Conspiracy labels
#     conspiracy_labels = Counter(item.get('conspiracy') for item in data)
#     print(f"\n🏷️  CONSPIRACY LABELS:")
#     for label, count in conspiracy_labels.most_common():
#         pct = count / len(data) * 100
#         print(f"   {str(label):12s}: {count:5d} ({pct:5.2f}%)")

#     # Quality checks
#     print(f"\n🔍 QUALITY CHECKS:")

#     issues = []

#     # Check 1: Empty texts
#     empty_texts = sum(1 for item in data if not item.get('text', '').strip())
#     if empty_texts > 0:
#         issues.append(f"❌ {empty_texts} empty texts")
#     else:
#         print(f"   ✅ No empty texts")

#     # Check 2: Markers without text
#     markers_no_text = sum(1 for item in data if len(item.get('markers', [])) > 0 and not item.get('text'))
#     if markers_no_text > 0:
#         issues.append(f"❌ {markers_no_text} items have markers but no text")
#     else:
#         print(f"   ✅ All markers have associated text")

#     # Check 3: Marker indices out of bounds
#     invalid_markers = 0
#     for item in data:
#         text = item.get('text', '')
#         text_len = len(text)
#         for marker in item.get('markers', []):
#             start = marker.get('startIndex', 0)
#             end = marker.get('endIndex', 0)
#             if start < 0 or end > text_len or start >= end:
#                 invalid_markers += 1

#     if invalid_markers > 0:
#         issues.append(f"❌ {invalid_markers} markers have invalid indices")
#     else:
#         print(f"   ✅ All marker indices are valid")

#     # Check 4: Unknown marker types
#     valid_types = {"Actor", "Action", "Effect", "Evidence", "Victim"}
#     unknown_types = set(marker_types.keys()) - valid_types
#     if unknown_types:
#         issues.append(f"❌ Unknown marker types: {unknown_types}")
#     else:
#         print(f"   ✅ All marker types are valid")

#     # Check 5: Conspiracy label coverage
#     if has_conspiracy < len(data) * 0.9:
#         issues.append(f"⚠️  Only {has_conspiracy/len(data)*100:.1f}% have conspiracy labels")
#     else:
#         print(f"   ✅ Good conspiracy label coverage")

#     # Summary
#     print(f"\n" + "="*70)
#     if issues:
#         print("⚠️  ISSUES FOUND:")
#         for issue in issues:
#             print(f"   {issue}")
#     else:
#         print("✅ NO ISSUES FOUND - Data looks good!")
#     print("="*70)

#     # Sample entries
#     print(f"\n📝 SAMPLE ENTRIES (First 2):")
#     for i, item in enumerate(data[:2], 1):
#         print(f"\n{i}. ID: {item.get('_id', 'N/A')}")
#         print(f"   Text: {item.get('text', '')[:100]}...")
#         print(f"   Conspiracy: {item.get('conspiracy', 'N/A')}")
#         print(f"   Markers: {len(item.get('markers', []))}")
#         if item.get('markers'):
#             for j, marker in enumerate(item['markers'][:2], 1):
#                 print(f"      {j}. {marker.get('type')}: \"{marker.get('text', '')}\"")


# if __name__ == "__main__":
#     print("\n" + "="*70)
#     print("DATA DIAGNOSTIC TOOL")
#     print("="*70)

#     # Diagnose train
#     diagnose_dataset("train_rehydrated.jsonl")

#     # Diagnose dev
#     diagnose_dataset("dev_rehydrated.jsonl")

#     print("\n" + "="*70)
#     print("DIAGNOSTIC COMPLETE!")
#     print("="*70)
#     print("\nIf you see issues above, fix them before training.")
#     print("If all looks good, proceed with training!")
#     print("="*70 + "\n")